# Семинар 6. Пошаговое руководство по популярным моделям и реализациям

**Преподаватель:** Никита Киселев

## Введение

На этом семинаре мы
- Рассмотрим особенности практической реализации современных диффузионных моделей.
- Поговорим о DDIM как ускоренном методе сэмплирования для DDPM.
- Обсудим, что еще придумали в последнее время для повышения качества диффузионных моделей.

**План занятия:**

1. DDIM как метод ускорения сэмплирования в DDPM.
2. Различные планировщики шума: примеры от базовых до продвинутых.
3. Использование нескольких text encoders для повышения качества восприятия текстовой информации (на примере SDXL и SD3).
4. Трансформерные архитектуры в диффузионных моделях на примере DiT, а также имплементация одного слоя MM-DiT из SD3.


## 1. DDIM

Сегодняшнее наше занятие мы начнем с очень важной темы в диффузионных моделях — Denoising Diffusion Implicit Models (DDIM).

Для начала давайте кратко пробежимся по тому, как устроены Denoising Diffusion **Probabilistic** Models (DDPM), которые вам уже прекрасно известны.

### 1.1. Воспоминание о DDPM

1. Последовательные прямой и обратный процессы являются **марковскими** процессами:
$$
q(\mathbf{x}_t | \mathbf{x}_{t-1}) = q(\mathbf{x}_t | \mathbf{x}_{t-1}, \mathbf{x}_0),
$$
то есть распределение каждого следующего изображения зависит только от предыдущего, и не от каких более ранних.

2. **Переходное распределение** в прямом процессе задается следующим образом:
   $$
        q(\mathbf{x}_t | \mathbf{x}_{t-1}) = \mathcal{N}\left( \sqrt{1 - \beta_t} \mathbf{x}_{t-1}, \beta_t \mathbf{I} \right),
   $$
   где $\{ \beta_t \in (0, 1) \}_{t=1}^{T}$ и $\beta_1 \leq \beta_2 \leq \ldots \leq \beta_T$ — расписание дисперсий. То есть постепенно, шаг за шагом, мы зашумляем исходное изображение вплоть до случайного шума. Также, вводя для простоты обозначений $\alpha_t = 1 - \beta_t$, его можно переписать в виде
   $$
        q(\mathbf{x}_t | \mathbf{x}_{t-1}) = \mathcal{N}\left( \sqrt{\alpha_t} \mathbf{x}_{t-1}, (1 - \alpha_t) \mathbf{I} \right).
   $$
   Здесь получается, что каждое новое изображение будет в среднем по амплитуде отличаться от предыдущего в $\sqrt{\alpha_t}$ раз и при этом будет еще более шумным, с дисперсией $(1 - \alpha_t)$.

3. Полезным также является выражение зашумленного на $t$-м шаге изображения через исходное:
   $$
        q(\mathbf{x}_t | \mathbf{x}_0) = \mathcal{N}\left( \sqrt{\bar{\alpha}_t} \mathbf{x}_0, (1 - \bar{\alpha}_t) \mathbf{I} \right),
   $$
   где $\bar{\alpha}_t = \prod_{i=1}^{t} \alpha_i$, то есть $\bar{\alpha}_t$ является произведением всех предыдущих коэффициентов и постепенно стремится к нулю при увеличении номера шага $t$.

4. Наконец, напомним выражение для обратного процесса (с учетом исходного изображения $\mathbf{x}_0$), которое также является гауссовским, со средним значением $\tilde{\boldsymbol{\mu}}(\mathbf{x}_t, \mathbf{x}_0)$, зависящим от зашумленного изображения $\mathbf{x}_t$ и исходного изображения $\mathbf{x}_0$, и дисперсией $\tilde{\sigma}_t^2$ (их можно честно вывести, используя формулу Байеса):
   $$
        q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) = \mathcal{N}\left( \tilde{\boldsymbol{\mu}}(\mathbf{x}_t, \mathbf{x}_0), \tilde{\sigma}_t^2 \mathbf{I} \right),
   $$
   где
   $$
        \tilde{\boldsymbol{\mu}}(\mathbf{x}_t, \mathbf{x}_0) = \frac{\sqrt{{\alpha}_t} (1 - \bar{\alpha}_{t-1})}{1 - \bar{\alpha}_t} \mathbf{x}_t + \frac{\sqrt{\bar{\alpha}_{t-1}} \beta_t}{1 - \bar{\alpha}_t} \mathbf{x}_0,
   $$
   $$
        \tilde{\sigma}_t^2 = \frac{1 - \bar{\alpha}_{t-1}}{1 - \bar{\alpha}_t} \beta_t,
   $$
   или же можно переписать среднее в другом виде, не через исходное изображение $\mathbf{x}_0$, а через шум $\boldsymbol{\epsilon}_t$:
   $$
        \tilde{\boldsymbol{\mu}}(\mathbf{x}_t, \boldsymbol{\epsilon}_t) = \frac{1}{\sqrt{\alpha_t}} \left( \mathbf{x}_t - \frac{1 - \alpha_t}{\sqrt{1 - \bar{\alpha}_t}} \boldsymbol{\epsilon}_t \right),
   $$
   где
   $$
        \boldsymbol{\epsilon}_t = \frac{1}{\sqrt{1 - \bar{\alpha}_t}} \left( \mathbf{x}_t - \sqrt{\bar{\alpha}_t} \mathbf{x}_0 \right).
   $$

5. В итоге мы обучаем одну из трех моделей (как раз-таки нейронных сетей) для вариационного распределения $p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t)$:

1) Предсказание среднего значения $\tilde{\boldsymbol{\mu}}(\mathbf{x}_t, \mathbf{x}_0)$ с помощью нейронной сети $\boldsymbol{\mu}_\theta(\mathbf{x}_t, t)$:
$$
    p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t) = \mathcal{N}\left( \boldsymbol{\mu}_\theta(\mathbf{x}_t, t), \tilde{\sigma}_t^2 \mathbf{I} \right).          
$$

2) Предсказание исходного изображения $\mathbf{x}_0$ с помощью нейронной сети $\hat{\mathbf{x}}_\theta(\mathbf{x}_t, t)$:
$$
    p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t) = \mathcal{N}\left( \frac{\sqrt{{\alpha}_t} (1 - \bar{\alpha}_{t-1})}{1 - \bar{\alpha}_t} \mathbf{x}_t + \frac{\sqrt{\bar{\alpha}_{t-1}} \beta_t}{1 - \bar{\alpha}_t} \hat{\mathbf{x}}_\theta(\mathbf{x}_t, t), \tilde{\sigma}_t^2 \mathbf{I} \right).         
$$

3) Предсказание шума $\boldsymbol{\epsilon}_t$ с помощью нейронной сети $\hat{\boldsymbol{\epsilon}}_\theta(\mathbf{x}_t, t)$:
$$
    p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t) = \mathcal{N}\left( \frac{1}{\sqrt{\alpha_t}} \left( \mathbf{x}_t - \frac{1 - \alpha_t}{\sqrt{1 - \bar{\alpha}_t}} \hat{\boldsymbol{\epsilon}}_\theta(\mathbf{x}_t, t) \right), \tilde{\sigma}_t^2 \mathbf{I} \right).         
$$

**Вопрос.** Имея зашумленную картинку $\mathbf{x}_t$ и предсказанный шум $\hat{\boldsymbol{\epsilon}}_t(\mathbf{x}_t, t)$, как мы можем получить предсказание $\mathbf{x}_0$? (Обозначим его $\mathbf{x}_{0|t}$.)

<details>
  <summary><b>Ответ</b></summary>

  <font color='green'>$\mathbf{x}_{0|t} = \frac{1}{\sqrt{\bar{\alpha}_t}} \left( \mathbf{x}_t - \sqrt{1 - \bar{\alpha}_t} \hat{\boldsymbol{\epsilon}}_\theta(\mathbf{x}_t, t) \right).$

  <b>Решение.</b> Получаем ответ напрямую из формулы $\boldsymbol{\epsilon}_t = \frac{1}{\sqrt{1 - \bar{\alpha}_t}} \left( \mathbf{x}_t - \sqrt{\bar{\alpha}_t} \mathbf{x}_0 \right)$.
  </font>
</details>

А теперь давайте подумаем о том, какие есть плюсы и минусы у DDPM:

$+$ Качественные сгенерированные изображения

$+$ Разнообразие объектов

$-$ Очень медленная скорость генерации

**Вопрос.** Как **ускорить** обратный процесс, при этом сохранив высокое качество генераций?

**Ответ.** А для этого и придумали DDIM!

### 1.2. Введение в DDIM

**Вопрос.** Какая была отличительная особенность последовательных прямого и обратного процессов в DDPM?

<details>
  <summary><b>Ответ</b></summary>
  
  <font color='green'>
  
  Они были марковскими, то есть на примере прямого процесса: $q(\mathbf{x}_t | \mathbf{x}_{t-1}) = q(\mathbf{x}_t | \mathbf{x}_{t-1}, \mathbf{x}_0)$.
  </font>
</details>

Авторы [Denoising Diffusion Implicit Models (DDIM)](https://arxiv.org/abs/2010.02502) предложили использовать **не марковский** прямой процесс! А какой же тогда? Давайте разбираться.

Прямой марковский процесс в DDPM выглядит вот так:

$$
    q(\mathbf{x}_{1:T} | \mathbf{x}_0) = \prod_{t=1}^{T} q(\mathbf{x}_{t} | \mathbf{x}_{t-1}).
$$

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1p-jML32gx9C-8bKuMvLKN5juBHU9gZzT" alt="ddpm" width="500"/>
    <figcaption> Прямой процесс DDPM. Источник: <a href="https://arxiv.org/abs/2006.11239">Ho et al. 2020</a> </figcaption>
</figure>

А вот авторы DDIM предложили построить процесс другим образом, введя

$$
    q_\sigma(\mathbf{x}_{1:T} | \mathbf{x}_0) = q_\sigma(\mathbf{x}_T | \mathbf{x}_0) \prod_{t=2}^{T} q_\sigma(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0).
$$

> **Комментарий.** В отличие от предыдущей формулы, здесь появляется дополнительный индекс $\sigma$. Как мы увидим позже, он означает наличие дополнительного набора параметров — расписания дисперсий $\sigma_1, \ldots, \sigma_T$.

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1T3SV7kc8b-Tbr_EVLpdaKNUYAPmw-EFY" alt="ddim" width="500"/>
    <figcaption> Прямой процесс DDIM. Источник: <a href="https://arxiv.org/abs/2006.11239">Ho et al. 2020</a> </figcaption>
</figure>

Таким образом,
- Последний в цепочке объект $\mathbf{x}_T$ получается напрямую из $\mathbf{x}_0$
- Каждый промежуточный зашумленный объект $\mathbf{x}_{t-1}$ получается из исходного $\mathbf{x}_0$ и следующего за ним $\mathbf{x}_t$!

Сразу же появляется вопрос, а как тогда определить $q_\sigma(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0)$?

Давайте отталкиваться от того, что мы уже видели, а именно рассмотрим Гауссовское распределение. Причем важно, что мы сделаем среднее как линейную функцию от $\mathbf{x}_0$ и $\mathbf{x}_t$. Запишется это требование следующим образом:
$$
    q_\sigma(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) = \mathcal{N}\left( w_0 \mathbf{x}_0 + w_t \mathbf{x}_t + b, \sigma_t^2 \mathbf{I} \right)
$$

**Вопрос.** Как определить $w_0$, $w_t$ и $b$?

<details>
  <summary><b>Ответ</b></summary>
  
  <font color='green'>

  Идея в том, чтобы сделать $q_\sigma(\mathbf{x}_t | \mathbf{x}_0)$ таким же, как было в DDPM, т.е.
$$
    q_\sigma(\mathbf{x}_t | \mathbf{x}_0) = \mathcal{N}\left( \sqrt{\bar{\alpha}_t} \mathbf{x}_0, (1 - \bar{\alpha}_t) \mathbf{I} \right)
$$
  </font>
</details>

### 1.3. Вывод обратного процесса в DDIM

Вывод будем производить последовательно, а именно пусть мы уже доказали, что
$$
    q_\sigma(\mathbf{x}_t | \mathbf{x}_0) = \mathcal{N}\left( \sqrt{\bar{\alpha}_t} \mathbf{x}_0, (1 - \bar{\alpha}_t) \mathbf{I} \right),
$$
как тогда подобрать $w_0$, $w_t$ и $b$, чтобы обеспечить
$$
    q_\sigma(\mathbf{x}_{t-1} | \mathbf{x}_0) = \mathcal{N}\left( \sqrt{\bar{\alpha}_{t-1}} \mathbf{x}_0, (1 - \bar{\alpha}_{t-1}) \mathbf{I} \right)?
$$

**Идея 1.** Обусловим распределение на $\mathbf{x}_t$ и проинтегрируем по нему, формально ничего не потеряв и не добавив:
$$
    q_\sigma(\mathbf{x}_{t-1} | \mathbf{x}_0) = \int q_\sigma(\mathbf{x}_{t-1}, \mathbf{x}_t | \mathbf{x}_0) d\mathbf{x}_t = \int q_\sigma(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) q_\sigma(\mathbf{x}_{t} | \mathbf{x}_0) d\mathbf{x}_t.
$$

**Идея 2.** Будем использовать интересное свойство нормального распределения. А именно, если $p(\mathbf{x}) = \mathcal{N}(\boldsymbol{\mu}, \sigma_{\mathbf{x}}^2\mathbf{I})$ и $p(\mathbf{y} | \mathbf{x}) = \mathcal{N}(a\mathbf{x} + b, \sigma_{\mathbf{y}}^2\mathbf{I})$, то
$$
    p(\mathbf{y}) = \int p(\mathbf{y}, \mathbf{x}) d\mathbf{x} = \int p(\mathbf{y} | \mathbf{x}) p(\mathbf{x}) d\mathbf{x} = \mathcal{N}\left(a \boldsymbol{\mu} + b, \left( \sigma_{\mathbf{y}}^2 + a^2 \sigma_{\mathbf{x}}^2 \right) \mathbf{I}\right).
$$

Воспользуемся **Идеей 2** для того, что мы получили в **Идее 1**. Тогда сразу же получим
$$
    q_\sigma(\mathbf{x}_{t-1} | \mathbf{x}_0) = \mathcal{N}\left( w_0 \mathbf{x}_0 + w_t (\sqrt{\bar{\alpha}}_t \mathbf{x}_0) + b, \left( \sigma_t^2 + w_t^2 (1 - \bar{\alpha}_t) \right) \mathbf{I}\right).
$$

**Вопрос.** Как дальше нужно действовать, чтобы подобрать коэффициенты $w_0$, $w_t$ и $b$?

<details>
  <summary><b>Ответ</b></summary>
  
  <font color='green'>Просто приравниваем! При этом начинаем с дисперсии.</font>

  <font color='green'>

1. Сначала приравниваем дисперсию, т.е.
$$
    \sigma_t^2 + w_t^2 (1 - \bar{\alpha}_t) = 1 - \bar{\alpha}_{t-1},
$$
откуда выражаем $w_t$ через все остальное:
$$
    w_t = \sqrt{\frac{1 - \bar{\alpha}_{t-1} - \sigma_t^2}{1 - \bar{\alpha}_t}}.
$$ </font>

  <font color='green'>

2. А теперь приравниваем средние и подставляем уже найденный $w_t$:
$$
    w_0 \mathbf{x}_0 + w_t (\sqrt{\bar{\alpha}}_t \mathbf{x}_0) + b =  \sqrt{\bar{\alpha}_{t-1}} \mathbf{x}_0,
$$
причем для простоты можем приравнять $b = 0$, откуда сразу легко следует
$$
    w_0 = \sqrt{\bar{\alpha}_{t-1}} - \sqrt{\bar{\alpha}_t} \sqrt{\frac{1 - \bar{\alpha}_{t-1} - \sigma_t^2}{1 - \bar{\alpha}_t}}.
$$</font>
  
</details>

Таким образом, мы получили формулу обратного процесса в DDIM:
$$
    q_\sigma(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) = \mathcal{N}\left( \sqrt{\bar{\alpha}_{t-1}} \mathbf{x}_0 - \sqrt{\bar{\alpha}_t} \sqrt{\frac{1 - \bar{\alpha}_{t-1} - \sigma_t^2}{1 - \bar{\alpha}_t}} \mathbf{x}_0 + \sqrt{\frac{1 - \bar{\alpha}_{t-1} - \sigma_t^2}{1 - \bar{\alpha}_t}} \mathbf{x}_t, \sigma_t^2 \mathbf{I} \right),
$$
или, наконец, финальное выражение:
$$
    q_\sigma(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) = \mathcal{N}\left( \sqrt{\bar{\alpha}_{t-1}} \mathbf{x}_0 + \sqrt{1 - \bar{\alpha}_{t-1} - \sigma_t^2} \cdot \frac{\mathbf{x}_t - \sqrt{\bar{\alpha}_t} \mathbf{x}_0}{\sqrt{1 - \bar{\alpha}_t}}, \sigma_t^2 \mathbf{I} \right),
$$
причем, подчеркнем, **при любых** $\sigma_t^2$ гарантированно $q_\sigma(\mathbf{x}_t | \mathbf{x}_0)$ остается таким же, как в DDPM!

Теперь давайте еще раз взглянем на DDPM и DDIM со стороны:

**DDPM**

- $q(\mathbf{x}_t | \mathbf{x}_{t-1})$ определено;
- $\color{purple}{q(\mathbf{x}_t | \mathbf{x}_0)}$ и $\color{olive}{q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0)}$ выводятся из $q(\mathbf{x}_t | \mathbf{x}_{t-1})$.

**DDIM**

- $\color{olive}{q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0)}$ определено;
- $\color{purple}{q(\mathbf{x}_t | \mathbf{x}_0)}$ и $q(\mathbf{x}_t | \mathbf{x}_{t-1})$ выводятся из $\color{olive}{q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0)}$.

**Вопрос.** Что будет, если мы сделаем $\sigma_t^2 = 0$?

<details>
  <summary><b>Ответ</b></summary>

  <font color='green'>Прямой и обратный процессы станут детерминистичными!</font>

</details>

А теперь давайте подумаем, что это означает и почему это может быть хорошо.

1. Для **одинаковых** $\mathbf{x}_T \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$ мы всегда получаем **один и тот же** $\mathbf{x}_0$.

2. Благодаря такой консистентности DDIM, мы можем осуществлять **семантическую интерполяцию** между изображениями, манипулируя именно **латентными переменными**, то есть тем шумом, с которого мы стартуем.

3. Как мы обсудим позже, детерминистичность DDIM также позволяет использовать **меньшее число шагов сэмплирования без потери качества**! А это критически важно для диффузионных моделей, которые в оригинале (DDPM) имеют очень медленное сэмплирование.

4. Наконец, в продолжение предыдущего пункта, если мы начинаем с одного и того же шума $\mathbf{x}_T \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$ и генерируем сэмплы с **различным числом шагов**, например 50 и 1000, то эти сэмплы практически **не будут отличаться по семантике**.

### 1.4. Обучение DDIM

**Вопрос.** Как Вам кажется, нужно ли для каждого выбранного расписания дисперсий $\{\sigma_1, \ldots, \sigma_T\}$ обучать отдельную модель DDIM?

<details>
  <summary><b>Ответ</b></summary>

<font color='green'> Оказывается, что нет! И эта особенность позволяет использовать DDIM намного проще. Мало того, вся прелесть DDIM заключается в том, что на самом деле его **обучение ничем не отличается** от обучения DDPM! Именно такую теорему доказывают авторы статьи. </font>

<font color='green'> Если говорить конкретно, то можно честно выписать и положить перед собой следующие две формулы:
- Функция потерь $L_{\gamma}(\hat{\boldsymbol{\epsilon}}_\theta)$ в модели DDPM:
$$
    L_{\gamma}(\hat{\boldsymbol{\epsilon}}_\theta) = \sum_{t=1}^{T} \gamma_t \mathbb{E}_{\mathbf{x}_0 \sim q(\mathbf{x}_0), \boldsymbol{\epsilon}_t \sim \mathcal{N}(\mathbf{0}, \mathbf{I})} \left[ \left\| \hat{\boldsymbol{\epsilon}}_{\boldsymbol{\theta}}(\sqrt{\alpha_t} \mathbf{x}_0 + \sqrt{1 - \alpha_t} \boldsymbol{\epsilon}_t, t) - \boldsymbol{\epsilon}_t \right \|_2^2 \right].
$$
- Функция потерь $J_{\sigma}(\hat{\boldsymbol{\epsilon}}_\theta)$ в модели DDIM:
$$
    L_{\gamma}(\hat{\boldsymbol{\epsilon}}_\theta) \equiv \sum_{t=1}^{T} \frac{1}{2 d \sigma_t^2 \alpha_t} \mathbb{E}_{\mathbf{x}_0 \sim q(\mathbf{x}_0), \boldsymbol{\epsilon}_t \sim \mathcal{N}(\mathbf{0}, \mathbf{I})} \left[ \left\| \hat{\boldsymbol{\epsilon}}_{\boldsymbol{\theta}}(\sqrt{\alpha_t} \mathbf{x}_0 + \sqrt{1 - \alpha_t} \boldsymbol{\epsilon}_t, t) - \boldsymbol{\epsilon}_t \right \|_2^2 \right],
$$</font>
<font color='green'>где мы обозначили за $d$ размерность $\mathbf{x}$, а знак $\equiv$ означает отбрасывание слагаемых, не зависящих от $\hat{\boldsymbol{\epsilon}}_\theta$.</font>

<font color='green'>

И можно видеть прекрасную вещь: действительно, полагая
$$
    \gamma_t = \frac{1}{2 d \sigma_t^2 \alpha_t},
$$
мы можем увидеть, что
$$
    J_{\sigma} = L_{\gamma} + \mathrm{const}.
$$</font>

<font color='green'>И тут мы понимаем, что нам **не нужно переучивать предсказатель шума**. Следовательно, предсказатель шума $\hat{\boldsymbol{\epsilon}}_\theta(\mathbf{x}_t, t)$, обученный для DDPM, может быть напрямую использован в обратном процессе DDIM! </font>
</details>

### 1.5. Обратные процессы в DDPM и DDIM

А теперь давайте еще раз пробежимся по тому, как выглядят обратные процессы в обоих методах.

**DDPM**

Формула обратного процесса:

$$
    q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) = \mathcal{N}\left( \frac{\sqrt{{\alpha}_t} (1 - \bar{\alpha}_{t-1})}{1 - \bar{\alpha}_t} \mathbf{x}_t + \frac{\sqrt{\bar{\alpha}_{t-1}} \beta_t}{1 - \bar{\alpha}_t} \mathbf{x}_0, \left( \frac{1 - \bar{\alpha}_{t-1}}{1 - \bar{\alpha}_t} \beta_t \right) \mathbf{I} \right)          
$$

Для каждого шага $t = T, \ldots, 1$, повторяем:

1. Подсчитать $\mathbf{x}_{0|t} = \frac{1}{\sqrt{\bar{\alpha}_t}} \left( \mathbf{x}_t - \sqrt{1 - \bar{\alpha}_t} \hat{\boldsymbol{\epsilon}}_\theta(\mathbf{x}_t, t) \right)$
2. Подсчитать $\color{purple}{\tilde{\boldsymbol{\mu}}(\mathbf{x}_t, \mathbf{x}_0) = \frac{\sqrt{{\alpha}_t} (1 - \bar{\alpha}_{t-1})}{1 - \bar{\alpha}_t} \mathbf{x}_t + \frac{\sqrt{\bar{\alpha}_{t-1}} \beta_t}{1 - \bar{\alpha}_t} \mathbf{x}_{0|t}}$
3. Просэмплировать $\mathbf{z}_t \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$
4. Подсчитать $\mathbf{x}_{t-1} = \tilde{\boldsymbol{\mu}}(\mathbf{x}_t, \mathbf{x}_0) + \color{olive}{\sqrt{\frac{1 - \bar{\alpha}_{t-1}}{1 - \bar{\alpha}_t} \beta_t}} \mathbf{z}_t$

**DDIM**

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1dAr0q1UUXoL1E7TVKrvvE9URA0XYR9D_" alt="ddim" width="700"/>
    <figcaption> Обратный процесс DDIM. Источник: <a href="https://arxiv.org/abs/2006.11239">Ho et al. 2020</a> </figcaption>
</figure>

Формула обратного процесса:

$$
    q_\sigma(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) = \mathcal{N}\left( \sqrt{\bar{\alpha}_{t-1}} \mathbf{x}_0 + \sqrt{1 - \bar{\alpha}_{t-1} - \sigma_t^2} \cdot \frac{\mathbf{x}_t - \sqrt{\bar{\alpha}_t} \mathbf{x}_0}{\sqrt{1 - \bar{\alpha}_t}}, \sigma_t^2 \mathbf{I} \right)
$$

Для каждого шага $t = T, \ldots, 1$, повторяем:

1. Подсчитать $\mathbf{x}_{0|t} = \frac{1}{\sqrt{\bar{\alpha}_t}} \left( \mathbf{x}_t - \sqrt{1 - \bar{\alpha}_t} \hat{\boldsymbol{\epsilon}}_\theta(\mathbf{x}_t, t) \right)$
2. Подсчитать $\color{purple}{\tilde{\boldsymbol{\mu}}(\mathbf{x}_t, \mathbf{x}_0) = \sqrt{\bar{\alpha}_{t-1}} \mathbf{x}_{0|t} + \sqrt{1 - \bar{\alpha}_{t-1} - \sigma_t^2} \cdot \frac{\mathbf{x}_t - \sqrt{\bar{\alpha}_t} \mathbf{x}_{0|t}}{\sqrt{1 - \bar{\alpha}_t}}}$
3. Просэмплировать $\mathbf{z}_t \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$
4. Подсчитать $\mathbf{x}_{t-1} = \tilde{\boldsymbol{\mu}}(\mathbf{x}_t, \mathbf{x}_0) + \color{olive}{\sigma_t} \mathbf{z}_t$

И на самом деле оказывается, что **DDIM — это обобщение DDPM**!

А именно, они совпадают, если
$$
    \sigma_t^2 = \frac{1 - \bar{\alpha}_{t-1}}{1 - \bar{\alpha}_t} \beta_t.
$$

В таком случае DDIM представляет собой марковский процесс!

### 1.6. Контроль стохастичности

Учитывая похожесть DDIM и DDPM, обычно вводят следующую величину, а именно перепараметризовывают
$$
    \sigma_t = \color{olive}{\eta} \sqrt{\frac{1 - \bar{\alpha}_{t-1}}{1 - \bar{\alpha}_t} \beta_t},
$$
тогда
- при $\eta = 0$ получаем детерминистичный процесс, то есть для одного и того же $\mathbf{x}_T \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$ мы будем всегда получать одинаковые $\mathbf{x}_0$;
- при $\eta = 1$ то же самое, что и DDPM.

А сейчас давайте вернемся к главной проблеме DDPM.

**Вопрос.** Что за проблема?

<details>
  <summary><b>Ответ</b></summary>
  
  <font color='green'>Очень медленное сэмплирование.</font>
</details>

### 1.7. Ускоренный процесс сэмплирования

Обратный процесс, с помощью которого производится сэмплирование как в DDPM, так и в DDIM, с последовательностью шагов $t \in [1, 2, \ldots, T]$ имеет следующий вид:
$$
    p_\theta(\mathbf{x}_{0:T}) = p_\theta(\mathbf{x}_T) \prod_{t=1}^T p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t).
$$

А как можно ускорить такой процесс? Давайте рассмотрим **подпоследовательность** временных шагов $\tau = [\tau_1, \tau_2, \ldots, \tau_S]$.

Тогда обратный процесс для подпоследовательности можно записать как
$$
    p_\theta(\mathbf{x}_\tau) = p_\theta(\mathbf{x}_T) \prod_{t=1}^{\color{olive}{S}} p_\theta(\mathbf{x}_{\color{olive}{\tau_{i-1}}} | \mathbf{x}_{\color{olive}{\tau_{i}}}).
$$

Возникает вопрос: в чем же была проблема использовать ускоренный процесс сэмплирования для DDPM?

Оказывется, что **маленькое число** шагов приводило к существенному **ухудшению качества** генерации.

Однако ухудшения качества **удалось избежать** при использовании DDIM, как только обратный процесс стал более **детерминистичным**. Интуитивно это можно объяснить для себя таким образом:
- Если мы используем **стохастичное сэмплирование**, то на каждом шаге мы добавляем случайный шум, причем если шагов мало, то и **шум будет больше каждый раз**.
- Поскольку наша модель предсказания шума все-таки не идеальна, складываясь каждый раз с добавленным шумом (см. предыдущий пункт), полученное предсказание будет становиться **все менее и менее точным**.
- В случае же **детерминистичного сэмплирования** накопления лишнего шума не происходит, потому мы и можем использовать меньшее число шагов **без потери качества**.

Давайте рассмотрим пример из статьи DDIM, который приводят авторы. Они взяли обученную модель DDPM и смотрели на то, как различные сэмплирования влияют на качество генераций.

Измерялась метрика FID (чем меньше, тем лучше) при варьировании
- $\eta$, отвечающей за стохастичность;
- $S$, отвечающей за число шагов в процессе сэмплирования.

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1y7RcQG-9Y5reM0M7x9xrK1VIGJNrz3G9" alt="ddpm-vs-ddim" width="500"/>
    <figcaption> Генерация изображений CIFAR10 и CelebA измеряется в FID. η=1 соответствует DDPM, а η=0 соответствует DDIM. Источник: <a href="https://arxiv.org/abs/2010.02502">Song et al. 2020</a> </figcaption>
</figure>

Из таблицы можно сделать следующие выводы:
- Когда $\eta = 1$ (DDPM), качество быстро ухудшается, когда $S$ уменьшается с 1000 до 10.
- Когда $\eta = 0$ (детерминистичный DDIM),
  - качество не такое плохое даже при $S = 10$;
  - качество при $S = 1000$ оказывается даже лучше, чем у DDPM!


### 1.8. Реализация обучения и сэмплирования DDIM

Как мы уже обсудили, обучение DDIM ничем не отличается от того же для DDPM. Поэтому мы рассмотрим пример кода, который был продемонстрирован на семинаре №3. Единственным отличием будет лишь реализация класса `NoiseScheduler`, который описывает процесс зашумления и расшумления модели.

Итак, давайте приступим. Для начала импортируем все необходимые библиотеки.

In [ ]:
import torch
from torch.nn import init
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset
from sklearn.datasets import make_moons
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

Разберемся с данными, на которых будем учить нейросеть. Для простоты выбираем совсем несложную задачу — будем аппроксимировать распределение точек на плоскости, которое внешне похоже на две луны.

In [ ]:
def moons_dataset(n=8000):
    X, _ = make_moons(n_samples=n, random_state=42, noise=0.03)
    X[:, 0] = (X[:, 0] + 0.3) * 2 - 1
    X[:, 1] = (X[:, 1] + 0.3) * 3 - 1
    return TensorDataset(torch.from_numpy(X.astype(np.float32)))

Провизуализируем данные, просэмплировав их из датасета.

In [ ]:
dataset = moons_dataset()
sample = torch.stack([sample[0] for sample in dataset])

plt.scatter(sample[:,0], sample[:,1], edgecolor="black", label="Real Data", color="green")
plt.grid(alpha=0.2)
plt.legend()
plt.tight_layout()
plt.show()

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1ujiiij0ZgcwHbpboaEXItXh3R8GRUune" alt="moons" width="400"/>
</figure>

Реализуем класс `PositionalEmbedding`, который будет создавать эмбеддинги для временного шага $t$ в нашем процессе. Напомним, что это нужно потому, что наша нейросеть $\hat{\boldsymbol{\epsilon}}_{\theta}(\mathbf{x}_t, t)$ имеет два входа: 1) зашумленный объект на шаге $t$; и 2) сам временной шаг $t$. Создание эмбеддингов нужно для того, чтобы модель лучше понимала текущее временное состояние.

In [ ]:
class SinusoidalEmbedding(nn.Module):
    """
    В качестве эмбеддингов будем использовать синусоидальные.
    """
    def __init__(self, size: int, scale: float = 1.0):
        super().__init__()
        self.size = size
        self.scale = scale

    def forward(self, x: torch.Tensor):
        x = x * self.scale
        half_size = self.size // 2
        emb = torch.log(torch.Tensor([10000.0])) / (half_size - 1)
        emb = torch.exp(-emb * torch.arange(half_size))
        emb = x.unsqueeze(-1) * emb.unsqueeze(0)
        emb = torch.cat((torch.sin(emb), torch.cos(emb)), dim=-1)
        return emb

    def __len__(self):
        return self.size


class PositionalEmbedding(nn.Module):
    def __init__(self, size: int, type: str, **kwargs):
        super().__init__()

        self.layer = SinusoidalEmbedding(size, **kwargs)

    def forward(self, x: torch.Tensor):
        return self.layer(x)

Теперь реализуем класс `MLP` непосредственно для нашей модели $\hat{\boldsymbol{\epsilon}}_\theta$. Поскольку данные, на которых мы будем обучать нашу модель, простые — точки на плоскости, мы возьмем очень простую архитектуру — буквально Multi Layer Perceptron (MLP).

In [ ]:
class Block(nn.Module):
    """
    Реализация одного блока внутри MLP.
    """
    def __init__(self, size: int):
        super().__init__()

        self.ff = nn.Linear(size, size)
        self.act = nn.GELU()

    def forward(self, x: torch.Tensor):
        return x + self.act(self.ff(x))


class MLP(nn.Module):
    """
    Модель для предсказания шума.
    """
    def __init__(self, hidden_size: int = 128, hidden_layers: int = 3, emb_size: int = 128,
                 time_emb: str = "sinusoidal", input_emb: str = "sinusoidal"):
        super().__init__()

        self.time_mlp = PositionalEmbedding(emb_size, time_emb)
        self.input_mlp1 = PositionalEmbedding(emb_size, input_emb, scale=25.0)
        self.input_mlp2 = PositionalEmbedding(emb_size, input_emb, scale=25.0)

        concat_size = len(self.time_mlp.layer) + \
            len(self.input_mlp1.layer) + len(self.input_mlp2.layer)
        layers = [nn.Linear(concat_size, hidden_size), nn.GELU()]
        for _ in range(hidden_layers):
            layers.append(Block(hidden_size))
        layers.append(nn.Linear(hidden_size, 2))
        self.joint_mlp = nn.Sequential(*layers)

    def forward(self, x, t):
        x1_emb = self.input_mlp1(x[:, 0])
        x2_emb = self.input_mlp2(x[:, 1])
        t_emb = self.time_mlp(t)
        x = torch.cat((x1_emb, x2_emb, t_emb), dim=-1)
        x = self.joint_mlp(x)
        return x

Наконец, перейдем к тому, что будет отличать нашу модель от той, что была на семинаре №3 — реализации процесса зашумления и расшумления. А именно, нам потребуются следующие изменения:
1. С DDIM мы можем контролировать стохастичность, используя параметр $\eta$. Добавим этот функционал в `__init__()`, вводя `eta`.
2. Добавим метод `set_timesteps()` для того, чтобы можно было изменять число шагов сэмплирования на инференсе.
3. Изменим метод `get_variance()`, в котором учтем, что шаги сэмплирования могут быть выбраны с пропусками.
4. В методе `q_posterior()` изменим формулу, чтобы соответствовать процессу DDIM.

In [ ]:
class NoiseScheduler():
    def __init__(self,
                 num_timesteps: int = 1000,
                 beta_start: float = 0.0001,
                 beta_end: float = 0.02,
                 beta_schedule: str = "linear",
                 eta: float = 0.0,
                 set_alpha_to_one: bool = True):

        self.num_timesteps = num_timesteps
        self.eta = eta

        if beta_schedule == "linear":
            self.betas = torch.linspace(
                beta_start, beta_end, num_timesteps, dtype=torch.float32)

        elif beta_schedule == "quadratic":
            self.betas = torch.linspace(
                beta_start ** 0.5, beta_end ** 0.5, num_timesteps, dtype=torch.float32) ** 2

        self.alphas = 1.0 - self.betas
        self.alphas_cumprod = torch.cumprod(self.alphas, axis=0)

        # required for self.add_noise
        self.sqrt_alphas_cumprod = self.alphas_cumprod ** 0.5
        self.sqrt_one_minus_alphas_cumprod = (1 - self.alphas_cumprod) ** 0.5

        # required for reconstruct_x0
        self.sqrt_inv_alphas_cumprod = torch.sqrt(1 / self.alphas_cumprod)
        self.sqrt_inv_alphas_cumprod_minus_one = torch.sqrt(
            1 / self.alphas_cumprod - 1)

        # At every step in DDIM, we are looking into the previous alphas_cumprod
        # For the final step, there is no previous alphas_cumprod because we are already at 0
        # `set_alpha_to_one` decides whether we set this parameter simply to one or
        # whether we use the final alpha of the "non-previous" one.
        self.final_alpha_cumprod = torch.tensor(1.0) if set_alpha_to_one else self.alphas_cumprod[0]

    def __len__(self):
        return self.num_timesteps

    def add_noise(self, x_start, x_noise, timestep):
        s1 = self.sqrt_alphas_cumprod[timestep]
        s2 = self.sqrt_one_minus_alphas_cumprod[timestep]

        s1 = s1.reshape(-1, 1)
        s2 = s2.reshape(-1, 1)

        return s1 * x_start + s2 * x_noise

    def reconstruct_x0(self, x_t, timestep, noise):
        s1 = self.sqrt_inv_alphas_cumprod[timestep]
        s2 = self.sqrt_inv_alphas_cumprod_minus_one[timestep]

        s1 = s1.reshape(-1, 1)
        s2 = s2.reshape(-1, 1)

        return s1 * x_t - s2 * noise

    def set_timesteps(self, num_inference_steps):
        self.num_inference_steps = num_inference_steps
        timesteps = (
            np.linspace(0, self.num_timesteps - 1, num_inference_steps)
            .round()[::-1]
            .copy()
            .astype(np.int64)
        )
        self.timesteps = torch.from_numpy(timesteps)

    def get_variance(self, timestep, prev_timestep):
        if timestep == 0:
            return 0
        alpha_prod_t = self.alphas_cumprod[timestep]
        alpha_prod_t_prev = self.alphas_cumprod[prev_timestep] if prev_timestep >= 0 else self.final_alpha_cumprod
        beta_prod_t = 1 - alpha_prod_t
        beta_prod_t_prev = 1 - alpha_prod_t_prev

        variance = (beta_prod_t_prev / beta_prod_t) * (1 - alpha_prod_t / alpha_prod_t_prev)
        variance = variance.clip(1e-20)
        return variance

    def q_posterior(self, pred_epsilon, sample, timestep, prev_timestep):
        alpha_prod_t = self.alphas_cumprod[timestep]
        alpha_prod_t_prev = self.alphas_cumprod[prev_timestep] if prev_timestep >= 0 else self.final_alpha_cumprod
        sqrt_one_minus_alphas_prod_t = self.sqrt_one_minus_alphas_cumprod[timestep]
        variance = self.get_variance(timestep, prev_timestep)
        sigma_t = self.eta * variance ** (0.5)

        pred_original_sample = self.reconstruct_x0(sample, timestep, pred_epsilon)
        pred_sample_direction = (1 - alpha_prod_t_prev - sigma_t**2) ** (0.5) * pred_epsilon
        mu = alpha_prod_t_prev ** (0.5) * pred_original_sample + pred_sample_direction

        return mu

    def step(self, model_output, timestep, sample):
        prev_timestep = timestep - self.num_timesteps // self.num_inference_steps
        pred_prev_sample = self.q_posterior(model_output, sample, timestep, prev_timestep)

        variance = 0
        if timestep > 0:
            noise = torch.randn_like(model_output)
            variance = self.get_variance(timestep, prev_timestep)
            sigma_t = self.eta * variance ** (0.5)
            variance = sigma_t * noise

        pred_prev_sample = pred_prev_sample + variance

        return pred_prev_sample

А сейчас, когда мы реализовали все, что нам было нужно, давайте перейдем к самому процессу обучения модели. Сперва зафиксируем гиперпараметры.

In [ ]:
NUM_SAMPLES_DATA = 10_000
BATCH_SIZE = 128

HIDDEN_SIZE = 128
HIDDEN_LAYERS = 3
EMBEDDING_SIZE = 128
TIME_EMBEDDING = "sinusoidal"
INPUT_EMEDDING = "sinusoidal"

NUM_TIMESTEPS = 50
NUM_INFERENCE_STEPS = NUM_TIMESTEPS
ETA = 0.0
BETA_SCHEDULE = 'linear'
LR = 5e-4

NUM_EPOCHS = 200

Задаем данные, модель и оптимизатор.

In [ ]:
dataset = moons_dataset(NUM_SAMPLES_DATA)
dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=BATCH_SIZE , shuffle=True, drop_last=True)

model = MLP(
        hidden_size=HIDDEN_SIZE,
        hidden_layers=HIDDEN_LAYERS,
        emb_size=EMBEDDING_SIZE,
        time_emb=TIME_EMBEDDING,
        input_emb=INPUT_EMEDDING)

noise_scheduler = NoiseScheduler(
        num_timesteps=NUM_TIMESTEPS,
        beta_schedule=BETA_SCHEDULE,
        eta=ETA)

optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=LR,
    )

Напомним, что обучение строится следующим образом:
- Берем сэмпл из реальных данных $\mathbf{x}$.
- Зашумляем сэмпл в соответствии со случайным моментом времени $t$.
- Предсказываем шум, которым он был зашумлен, используя нашу модель $\hat{\boldsymbol{\epsilon}}_\theta$.
- Подставляем полученное значение в функцию потерь и делаем шаг оптимизатора.

In [ ]:
global_step = 0
frames = []
losses = []

for epoch in tqdm(range(NUM_EPOCHS)):

    model.train()

    for step, batch in enumerate(dataloader):
        batch = batch[0]
        noise = torch.randn(batch.shape)
        timesteps = torch.randint(
            0, noise_scheduler.num_timesteps, (batch.shape[0],)
        ).long()

        noisy = noise_scheduler.add_noise(batch, noise, timesteps)
        noise_pred = model(noisy, timesteps)
        loss = F.mse_loss(noise_pred, noise)
        loss.backward(loss)

        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.detach().item())

Ура, мы обучили модель! А теперь давайте просэмплируем из нее, чтобы посмотреть на то, как мы аппроксимируем реальные данные.

In [ ]:
model.eval()
timesteps = list(range(len(noise_scheduler)))[::-1]
noise_scheduler.set_timesteps(len(noise_scheduler))
sample = torch.randn(1024, 2) # sampling from noise

for i, t in enumerate(tqdm(timesteps)):
    t = torch.from_numpy(np.repeat(t,  1024)).long()
    with torch.no_grad():
        residual = model(sample, t)
    sample = noise_scheduler.step(residual, t[0], sample)

In [ ]:
plt.scatter(sample[:,0], sample[:,1], edgecolor="black", label="Generated Data")
plt.grid(alpha=0.2)
plt.legend()
plt.tight_layout()
plt.show()

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1-BSvNVOf8Fcww0i4tfZEi7pW4yFHH0bN" alt="moons-50" width="400"/>
</figure>

Как видно, просэмплированные с помощью модели данные выглядят так же, как и реальные.

**Вопрос.** Что такого мы забыли пронаблюдать, что является ключевой возможностью модели DDIM?

<details>
  <summary><b>Ответ</b></summary>
  
  <font color='green'>Быстрое сэмплирование с пропуском шагов!</font>
</details>

Давайте же теперь реализуем и его. Для этого достаточно просто изменить `timesteps`, из которых мы берем каждый временной шаг `t`.

In [ ]:
NUM_INFERENCE_STEPS = 10

model.eval()
timesteps = list(range(NUM_INFERENCE_STEPS))[::-1]
noise_scheduler.set_timesteps(NUM_INFERENCE_STEPS)
sample = torch.randn(1024, 2) # sampling from noise

for i, t in enumerate(tqdm(timesteps)):
    t = torch.from_numpy(np.repeat(t,  1024)).long()
    with torch.no_grad():
        residual = model(sample, t)
    sample = noise_scheduler.step(residual, t[0], sample)

In [ ]:
plt.scatter(sample[:,0], sample[:,1], edgecolor='black', label="Generated Data")
plt.grid(alpha=0.2)
plt.legend()
plt.tight_layout()
plt.show()

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1dKwuwLLbbVEHYHUoKiVlsRdxNhNis2Xm" alt="moons-10" width="400"/>
</figure>

Вот так, сократив число шагов сэмплирования с 50 до 10, мы практически ничего не потеряли: данные опять похожи на реальные.

Таким образом, мы подробно рассмотрели DDIM:
- Какая идея стояла за выводом прямого и обратного процессов.
- Как получить обратный процесс.
- Как выглядит сэмплирование.
- В чем отличие от DDPM, и в чем схожесть.
- Ускоренный процесс сэмплирования с помощью пропуска шагов.
- Реализовали обучение модели DDIM и сэмплирование из нее на примере, разобранном в семинаре №3.

## 2. Планировщики шума в диффузионных моделях

Итак, мы уже поняли, что проблема с медленным сэмплированием в DDPM может быть решена за счет изменения процедуры получения $\mathbf{x}_{t-1}$ из $\mathbf{x}_t$ в обратном процессе, как это происходило в DDIM.
Идея DDIM оказалась крайне удачной в том числе и потому, что чисто математически обучение DDIM соответствует обучению DDPM.
Ведь в таком случае, поменяв всего лишь процесс расшумления, мы получаем возможность существенного ускорения инференса модели!

Однако остается открытым вопрос, можно ли как-то иначе ускорить процесс сэмплирования.
Итак, мы переходим к другим методам, позволяющим изменить процессы зашумления и расшумления, чтобы достичь ускорения процесса.

### 2.1. Терминология

Начнем с терминологии.

В контексте диффузионных моделей принято использовать такое понятие, как **scheduler** или планировщик (также известный как sampler/solver).

Планировщик — это некий алгоритм, который не содержит в себе обучаемых параметров. Он отвечает за то, как именно мы зашумляем изображение.

Подробно о планировщиках в генеративных моделях написано в [статье от NVLabs](https://arxiv.org/pdf/2206.00364.pdf), однако мы постараемся составить краткое их описание.

1. Итак, scheduler используется и на этапе обучения, и на этапе инференса для восстановления зашумленного изображения.
2. Глобально существует два вида планировщиков: с детерминистическим сэмплированием и со стохастическим:
   - Первый тип в $N$ шагов решает обыкновенное дифференциальное уравнение (ОДУ) динамики движения изображения к абсолютному шуму.
   - Второй тип работает лучше с точки зрения качества генерации. На каждом шаге планировщика к данным добавляется свежий шум, сэмплированный из некоторого распределения. В этом случае уже решается стохастическое дифференциальное уравнения (СДУ).
3. Планировщик в паре с нейронной сетью образуют цикл, в котором решается ДУ.

Таким образом, планировщик задает правила обработки шума. Различные планировщики имеют разные скорости шумоподавления и компромиссы в отношении качества.

**Вопрос.** Откуда два именно таких вида планировщиков: детерминистичные и стохастические?

<details>
  <summary><b>Ответ</b></summary>
  
  <font color='green'>Все кроется в математике непрерывного диффузионного процесса!</font>
</details>

Как бы нам ни хотелось упростить повествование, здесь не обойтись без небольшого напоминания непрерывного диффузионного процесса.

Начнем с того, что в целом процесс изменения во времени некоторого объекта $\mathbf{x}$ может быть описан двумя принципиально разными подходами.

**Обыкновенное дифференциальное уравнение (ОДУ)**

$$
    \frac{d\mathbf{x}(t)}{dt} = \mathbf{f}(\mathbf{x}(t), t), \qquad \mathbf{x}(0) = \mathbf{x}_0,
$$

где функция $\mathbf{f}(\mathbf{x}(t), t)$ задает динамику изменения объекта во времени.

**Стохастическое дифференциальное уравнение (СДУ)**

$$
    d\mathbf{x} = \mathbf{f}(\mathbf{x}, t) dt + g(t) d\mathbf{w}, \qquad \mathbf{x}(0) \sim p_0(\mathbf{x}) = \pi(\mathbf{x}),
$$

где функция $\mathbf{f}(\mathbf{x}(t), t)$ называется функцией **дрифта**, а $g(t)$ называется функцией **диффузии**.

При этом в таком процессе плотность вероятности $\mathbf{x}$ изменяется с течением времени согласно так называемому пути вероятности $p_t(\mathbf{x}) = p(\mathbf{x}, t)$.

Здесь мы видим одну особенность, которая отличает СДУ от ОДУ.

**Вопрос.** Что за слагаемое отличает СДУ от ОДУ?

<details>
  <summary><b>Ответ</b></summary>
  
  <font color='green'>Слагаемое $g(t) d\mathbf{w}$, которое вносит случайность в этот процесс.</font>
</details>

Здесь через $\mathbf{w}(t)$ мы обозначили стандартный винеровский процесс.

Напомним, что это такой процесс, у которого все приращения независимы и нормальны, то есть $\mathbf{w}(t) - \mathbf{w}(s) \sim \mathcal{N}(0, (t-s)\mathbf{I})$.

Математически легко показать, что $d\mathbf{w} = \mathbf{w}(t + dt) - \mathbf{w}(t) \sim \mathcal{N}(0, \mathbf{I} \cdot dt)$, то есть $d\mathbf{w} = \boldsymbol{\epsilon} \cdot \sqrt{dt}$, где $\boldsymbol{\epsilon} \sim \mathcal{N}(0, \mathbf{I})$.

**Вопрос.** Какое условие на функции дрифта и диффузии приводит СДУ к ОДУ?

<details>
  <summary><b>Ответ</b></summary>

  <font color='green'>$g(t) = 0$.</font>
</details>

А теперь вспомним из предыдущих занятий, что конкретный вид функций дрифта и диффузии приводит нас именно к диффузионному процессу DDPM, и этот вид следующий:

$$
    d\mathbf{x} = -\frac{1}{2} \beta(t) \mathbf{x}(t) dt + \sqrt{\beta(t)} \cdot d\mathbf{w},
$$
то есть функции дрифта и диффузии задаются как
$$
    \mathbf{f}(\mathbf{x}, t) = -\frac{1}{2} \beta(t) \mathbf{x}(t), \qquad g(t) = \sqrt{\beta(t)}.
$$

Последние несколько фактов, которые нам пригодятся, позволяют обратить прямой процесс диффузии (написанный выше) и получить обратные СДУ и ОДУ.

1. Для каждого СДУ вида $d\mathbf{x} = \mathbf{f}(\mathbf{x}, t) dt + g(t) d\mathbf{w}$ существует ОДУ с тем же самым путем вероятности $p_t(\mathbf{x})$:
   
$$
    d\mathbf{x} = \left( \mathbf{f}(\mathbf{x}, t) - \frac{1}{2} g^2(t) \nabla_\mathbf{x} \log p_t(\mathbf{x}) \right) dt.
$$

2. Для каждого СДУ вида $d\mathbf{x} = \mathbf{f}(\mathbf{x}, t) dt + g(t) d\mathbf{w}$ существует обратное СДУ:
$$
    d\mathbf{x} = \left( \mathbf{f}(\mathbf{x}, t) - \color{purple}{ g^2(t) \nabla_\mathbf{x} \log p_t(\mathbf{x})} \right) dt + \color{olive}{g(t) d\mathbf{w}}, \qquad dt < 0.
$$

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1lyIB_qGUevIu4JZ0ridJc2OyvkCzSCGR" alt="sde-inverse" width="600"/>
    <figcaption> Прямой и обратный диффузионные процессы как SDE. Источник: <a href="https://arxiv.org/abs/2011.13456">Song et al. 2020</a> </figcaption>
</figure>

То есть для нашего прямого процесса диффузии мы можем
1. Выписать обратный процесс диффузии, который тоже будет СДУ.
2. Для полученного обратного процесса выписать соотвествующее ему ОДУ.

**Вопрос.** Как это можно использовать, если мы уже обучили свою диффузионную модель?

<details>
  <summary><b>Ответ</b></summary>
  
  <font color='green'>Обратим внимание на то, что стоит в формуле обратного процесса, а именно на градиент логарифма плотности $\nabla_\mathbf{x} \log p_t(\mathbf{x})$. Это ведь и есть score-функция нашего зашумленного распределения изображения. И именно ее мы аппроксимируем, используя нейросеть $\hat{\boldsymbol{\epsilon}}_\theta$!</font>

  <font color='green'>Так что же теперь получается? Мы можем
1. Обучить нейронную сеть так же, как это было в DDPM.
2. Использовать ее для аппроксимации score-функции $\nabla_{\mathbf{x}} \log p_t(\mathbf{x})$.
3. Решая это СДУ (или соответствующее ОДУ), получать сэмпл $\mathbf{x}_0$.
</font>
</details>

Здесь хочется отметить то, как связаны процессы сэмлирования DDPM и DDIM с решением таких уравнений.

На самом деле, все довольно логично и красиво:
- Дискретизация обратного **СДУ** для дифффузионного процесса приводит нас к **DDPM-сэмплированию**.
- Дискретизация обратного **ОДУ** для дифффузионного процесса приводит нас к **DDIM-сэмплированию**.

Это очень хорошо соотносится с тем, что мы получили ранее, — что на самом деле DDIM лишь детерминистичная версия DDPM.

Оказывается, что люди уже рассмотрели много разных способов описания диффузионного процесса через СДУ.

Кроме того, существует уже и множество методов, позволяющих решать такие СДУ и ОДУ наиболее эффективно.

Теперь давайте перейдем к тому, что рассмотрим основные такие методы, которые чаще всего применяются на практике.

### 2.2. Таксономия планировщиков

Начнем с того, что распределим основные известные планировщики по нескольким категориям. Далее мы еще коснемся большинства из них подробнее.

#### 2.2.1. Old-School ODE solvers

Это самые простые планровщики, которые на самом деле были впервые предложены еще **несколько сотен лет назад**. Конечно, это было не в контексте диффузионных моделей, а для решения обыкновенных дифференциальных уравнений (ОДУ/ODE).

- Euler — самый что ни на есть простой солвер первого порядка.
- Heun — более точный, но медленнее Euler.
- LMS (Linear multi-step method) — такой же быстрый, как Euler, но обычно более точный.

**Важно** подметить, что все они являются сугубо **детерминистичными**.

Давайте в качестве примера разберем реализацию Euler и Heun, как довольно простых методов решения ОДУ.

Нас интересует уравнение следующего вида:
$$
    \frac{d\mathbf{x}(t)}{dt} = \mathbf{f}(\mathbf{x}(t), t).
$$

**Euler**

Первый метод, Euler, получается просто из разностной аппроксимации этого уравнения. Это метод первого порядка, что означает, что локальная ошибка разностной аппроксимации будет порядка $o(\Delta t)$.
- Заметим, что $d\mathbf{x} = \mathbf{x}(t + dt) - \mathbf{x}(t)$.
- Зафиксируем конечное **отрицательное** приращение по времени $dt = - \Delta t$ (отрицательное потому, что мы берем **обратный** процесс диффузии).
- Положим $\mathbf{x}(t) = \mathbf{x}_t$ и $\mathbf{x}(t + dt) = \mathbf{x}_{t-1}$ для дискретных шагов времени $t$.

Тогда мы получаем следующее правило обновления, которое и называется методом Эйлера (Euler):
$$
    \mathbf{x}_{t-1} = \mathbf{x}_t - \Delta t \cdot \mathbf{f}(\mathbf{x}_t, t).
$$

**Heun**

Второй метод, Heun, уже является **методом второго порядка**. Это означает, что локальная ошибка разностной аппроксимации будет порядка $o((\Delta t)^2)$.
- Для начала используем ту же разностную аппроксимацию, что и в методе Эйлера, тем самым получая приближение к $\mathbf{x}_{t-1}$.
- Однако на этом не останавливаемся и заменяем $\mathbf{f}(\mathbf{x}_t, t)$ на усредненное по двум точкам значение:

$$
\begin{aligned}
    \tilde{\mathbf{x}}_{t-1} &= \mathbf{x}_t - \Delta t \cdot \mathbf{f}(\mathbf{x}_t, t), \\
    \mathbf{x}_{t-1} &= \mathbf{x}_t - \frac{\Delta t}{2} \cdot \left[ \mathbf{f}(\mathbf{x}_t, t) + \mathbf{f}(\tilde{\mathbf{x}}_{t-1}, t-1) \right]. \\
\end{aligned}
$$

Перейдем к их реализации для решения простого дифференциального уравнения для экспоненты:
$$
    \frac{dx}{dt} = x, \qquad x(0) = 1.
$$

Его решением, как нетрудно заметить, является функция $x(t) = e^t$. Посмотрим, какое решение выдаст каждый из рассмотренных методов.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the ODE function
def f(t, x):
    return x

# Euler method
def euler_method(f, x0, t_range, h):
    x = [x0]
    t = [0]
    for i in range(1, t_range):
        x.append(x[-1] + h * f(t[-1], x[-1]))
        t.append(t[-1] + h)
    return t, x

# Heun method
def heun_method(f, x0, t_range, h):
    x = [x0]
    t = [0]
    for i in range(1, t_range):
        k1 = f(t[-1], x[-1])
        k2 = f(t[-1] + h, x[-1] + h * k1)
        x.append(x[-1] + h * (k1 + k2) / 2)
        t.append(t[-1] + h)
    return t, x

# Parameters
x0 = 1
t_range = 100
h = 0.1 # это шаг дискретизации

# Solve using Euler method
t_euler, x_euler = euler_method(f, x0, t_range, h)

# Solve using Heun method
t_heun, x_heun = heun_method(f, x0, t_range, h)

# Exact solution
t_exact = np.linspace(0, t_range * h, t_range)
x_exact = np.exp(t_exact)

# Plot the results
plt.figure(figsize=(6, 4))
plt.plot(t_euler, x_euler, label='Euler Method', linestyle='--')
plt.plot(t_heun, x_heun, label='Heun Method', linestyle='-.')
plt.plot(t_exact, x_exact, label='Exact Solution', linestyle='-')
plt.xlabel('Time (t)')
plt.ylabel('x(t)')
plt.title('Comparison of Euler and Heun Methods')
plt.legend()
plt.grid(True)
plt.show()

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=11Xt2uUJ0TpVZ6suPmC2rR26s7V29iEM9" alt="euler-heun" width="400"/>
</figure>

Можно видеть, что метод Euler отстает по качеству от метода Heun.

Аналогичным образом эти методы могут применяться и для решения более сложных дифференциальных уравнений, в частности и DDPM.

А теперь давайте перейдем к другим, более навороченным методам, на которых уже сфокусируемся менее детально.

#### 2.2.2. Ancestral samplers

Так называемые анкестральные (ancestral) сэмплеры. Идея в том, что, в отличие от обычных ODE solvers, они уже не являются детерминистичными за счет внесения шума в получаемый сэмпл. Причем этот шум вносится всегда, из-за чего такие солверы **не сходятся** (продолжают изменять изображение даже при большом числе шагов).

- Euler a
- DPM2 a
- DPM++ 2S a
- DPM++ 2S a Karras

Приписка Karras здесь отсылает нас к [статье](https://arxiv.org/abs/2206.00364), в которой авторы предложили делать расписание шума еще более маленьким в конце процесса.

#### 2.2.3. DDIM и PLMS

Ранее мы уже обсудили, что собой представляет DDIM. Отметим лишь, что PLMS — это более новая и быстрая альтернатива для DDIM. По большому счету, эти двое уже давно устарели и сейчас уже не так широко применяются.

#### 2.2.4. DPM и DPM++

DPM (Diffusion probabilistic model solver) и DPM++ представляют целое семейство солверов схожей архитектуры.

- DPM и DPM2 очень похожи между собой, разве что DPM2 является методом 2-го порядка (а потому более точный, но и более долгий).
- DPM++ является усовершенствованием DPM.
- DPM adaptive подстраивает размер шага адаптивным образом. Это может замедлить процесс, поскольку не гарантирует окончание за определенное число шагов сэмплирования.

#### 2.2.5. UniPC

UniPC (Unified Predictor-Corrector) был вдохновлен методом predictor-corrector, который широко известен и применим при решении ОДУ. Благодаря этому методу можно генерировать качественные изображения всего за 5—10 шагов.

### 2.3. Какой солвер выбрать?

> "Что лучше: Ferrari или Jeep?"
>
> Очевидно, ответ зависит от того, собираетесь вы ехать по бездорожью или нет, правда же?

Так и в вопросе о том, какой планировщик является самым лучшим. В зависимости от того, что вы хотите получить в итоге, вам может понадобиться использовать различные варианты. Рассмотрим основные наши желания.

#### 2.3.1. Качество изображения

Если вы стремитесь получить наилучшее качество итогового изображения, стоит обратить внимание на сходимость методов. Как ранее обсуждалось, анкестральные методы не сходятся, так что сразу их отметаем. Если Ввы при этом не хотите ждать несколько сотен шагов, то и DDIM тоже отпадает. Heun или LMS Karras обычно показывают хорошие результаты, но лучше использовать DPM++ 2M или его Karras-версию.

Вы также можете попробовать DPM adaptive, если никуда не спешите, или же UniPC, если все-таки экономите время.

С упомянутыми сэмплерами получится достичь хороших результатов генерации всего за 20—30 шагов.

#### 2.3.2. Скорость генерации

Если вы тестируете различные промпты и не хотите тратить много времени на ожидание, вам определенно стоит обратить внимание на DPM++ 2M или UniPC с небольшим числом шагов.

Всего 10—15 шагов хватит для получения довольно неплохих результатов.

Если вы все-таки не особенно волнуетесь за воспроизводимость, то можете попробовать и Euler A, быстрый и довольно качественный анкестральный сэмплер.

#### 2.3.3. Креативность и гибкость

Здесь хотелось бы отметить анкестральные и стохастичные сэмплеры. Их проблема (но одновременно и достоинство, смотря как воспринимать) заключается в том, что если на 40-м шаге вы получили хороший сэмпл, то на 50-м ничего не мешает получить такой же, а то и хуже. Эта лотерея делает такие сэмплеры более «креативными», поскольку вам определенно потребуется постоянно менять число шагов для получения желаемого результата.

Ну и, конечно, Euler A и DPM++ SDE Karras здесь особенно выделяются. Попробуйте сгенерировать изображения за 15, 20, 25 шагов и посмотрите, что будет получаться.

### 2.4. Реализация в коде

Как и большинство других методов для диффузионных моделей, планировщики широко представлены в библиотеке [🤗 Diffusers](https://huggingface.co/docs/diffusers/api/schedulers/overview).

Все schedulers наследуются от базового класса [`SchedulerMixin`](https://huggingface.co/docs/diffusers/v0.31.0/en/api/schedulers/overview#diffusers.SchedulerMixin), в котором имплементированы базовые низкоуровневые операции.

На выходе scheduler возвращает `SchedulerOutput`, который содержит в себе единственный аргумент — `prev_sample`, то есть сэмпл для предыдущего момента времени.

[`KarrasDiffusionSchedulers`](https://github.com/huggingface/diffusers/blob/a69754bb879ed55b9b6dc9dd0b3cf4fa4124c765/src/diffusers/schedulers/scheduling_utils.py#L32) представляют собой широкое обобщение сэмплеров из линейки 🤗 Diffusers. Солверы этого класса отличаются своей стратегией сэмплирования шума, типом сети и масштабирования, стратегией обучения и тем, как взвешивается лосс.

Различные солверы в этом классе, в зависимости от типа средства решения обыкновенных дифференциальных уравнений (ODE), подпадают под вышеуказанную таксономию и обеспечивают хорошую абстракцию для проектирования основных сэмплеров, реализованных в 🤗 Diffusers.

При этом нельзя сказать, что какой-то из них лучше других. Каждый из них имеет свои плюсы и минусы.

Лучший способ узнать, какой из них лучше всего подходит вам, — попробовать их.

In [ ]:
# Устанавливаем необходимые библиотеки и импортируем их
# !pip install diffusers transformers

import torch
from diffusers import (
    # Stable Diffusion
    StableDiffusionPipeline,
    # schedulers
    DPMSolverMultistepScheduler,
    DPMSolverSinglestepScheduler,
    KDPM2DiscreteScheduler,
    KDPM2AncestralDiscreteScheduler,
    EulerDiscreteScheduler,
    EulerAncestralDiscreteScheduler,
    HeunDiscreteScheduler,
    LMSDiscreteScheduler,
    DEISMultistepScheduler,
    UniPCMultistepScheduler,
)
from PIL import Image
from tqdm import tqdm
from diffusers.utils import make_image_grid

In [ ]:
# И определяем модель и Shedulers для тестирования
model_id = "stabilityai/stable-diffusion-2"
pipe = StableDiffusionPipeline.from_pretrained(model_id)

schedulers = [
    DPMSolverMultistepScheduler(),  # DPM++ 2M
    DPMSolverMultistepScheduler(use_karras_sigmas=True),  # DPM++ 2M Karras
    DPMSolverMultistepScheduler(algorithm_type="sde-dpmsolver++"),  # DPM++ 2M SDE
    DPMSolverMultistepScheduler(use_karras_sigmas=True, algorithm_type="sde-dpmsolver++"),  # DPM++ 2M SDE Karras
    DPMSolverSinglestepScheduler(),  # DPM++ SDE
    DPMSolverSinglestepScheduler(use_karras_sigmas=True),  # DPM++ SDE Karras
    KDPM2DiscreteScheduler(),  # DPM2
    KDPM2DiscreteScheduler(use_karras_sigmas=True),  # DPM2 Karras
    KDPM2AncestralDiscreteScheduler(), # DPM2 a
    KDPM2AncestralDiscreteScheduler(use_karras_sigmas=True), # DPM2 a Karras
    EulerDiscreteScheduler(),  # Euler
    EulerAncestralDiscreteScheduler(),  # Euler a
    HeunDiscreteScheduler(),  # Heun
    LMSDiscreteScheduler(),  # LMS
    LMSDiscreteScheduler(use_karras_sigmas=True),  # LMS Karras
]

Вы можете узнать, какие планировщики совместимы с текущей моделью, вызвав метод `compatibles`.

In [ ]:
pipe.scheduler.compatibles

А теперь перейдем к генерациям.

In [ ]:
# Задаем промпт и другие параметры для генерации
prompt = "a red-haired kitten sitting on a stone against the background of a green garden"
negative_prompt = "blurry, noisy, overexposure, artefacts"
guidance_scale = 5.0
num_inference_steps = 10

images = []

# Тестируем каждый планировщик
for scheduler in schedulers:

    # Получаем название
    scheduler_name = scheduler.__class__.__name__
    print(f"Testing {scheduler_name}...")

    # Переключаться между планировщиками довольно просто:
    pipe.scheduler = scheduler.from_config(pipe.scheduler.config)
    pipe = pipe.to("cuda")

    # Генерируем изображение, используя заданный sheduler
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        guidance_scale=guidance_scale,
        num_inference_steps=num_inference_steps,
        generator=torch.Generator(device='cpu').manual_seed(307),
    ).images[0]

    images.append(image)

# Выводим сетку изображений
make_image_grid(images, 3, 5)

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1JO55VWUwu23AMPRzcJq-W65ZtYJCKJ-X" alt="schedulers" width="800"/>
</figure>

Посмотрим на то, насколько сильно зависит каждый из сэмплеров от числа шагов расшумления.

In [ ]:
featured_schedulers = [
    DPMSolverMultistepScheduler(),  # DPM++ 2M
    DPMSolverSinglestepScheduler(),  # DPM++ SDE
    KDPM2DiscreteScheduler(),  # DPM2
    KDPM2AncestralDiscreteScheduler(), # DPM2 a
    EulerDiscreteScheduler(),  # Euler
    EulerAncestralDiscreteScheduler(),  # Euler a
    HeunDiscreteScheduler(),  # Heun
    LMSDiscreteScheduler(),  # LMS
]

In [ ]:
images = []

steps_list = [3, 5, 7, 10, 15, 20]

# Тестируем каждый scheduler
for scheduler in tqdm(featured_schedulers):

    # Подгружаем новый планировщик
    pipe.scheduler = scheduler.from_config(pipe.scheduler.config)
    pipe = pipe.to("cuda")

    for num_inference_steps in steps_list:

        # Генерируем изображение
        image = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            guidance_scale=guidance_scale,
            num_inference_steps=num_inference_steps,
            generator=torch.Generator(device='cpu').manual_seed(307),
        ).images[0]

        images.append(image)

# Выводим сетку из изображений
make_image_grid(images, len(featured_schedulers), len(steps_list))

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1f61ZoZL-yffQ1eGbRzmk2-xa3RFgSQCa" alt="schedulers-steps" width="800"/>
</figure>

- Каждая строка соответствует одному сэмплеру
- Каждый столбец соответствует определенному числу шагов расшумления на инференсе
- Можно видеть, что одним из лучших решений будет DPMSolver

## 3. Использование нескольких текстовых энкодеров (text encoders)

Следующий наш пункт в сегодняшнем обсуждении современных методов в диффузионных моделях — это использование нескольких текстовых энкодеров.

**Вопрос.** Как вы думаете: зачем использовать несколько текстовых энкодеров?

<details>
  <summary><b>Ответ</b></summary>
  
  <font color='green'>Каждый из энкодеров привносит свою особенную информацию о входной последовательности. В итоге, объединяя их выходы, можно получить более точное и полноценное описание.</font>
</details>

В нашем сегодняшнем разговоре мы остановимся на некоторых известных диффузионных моделях, в которых авторы предложили использовать несколько текстовых энкодеров.

### 3.1. SDXL

И первая статья, которая попадает под наше рассмотрение, — [Podell et al. *SDXL: Improving Latent Diffusion Models for High-Resolution Image Synthesis.*](https://arxiv.org/abs/2307.01952). Мы не будем надолго останавливаться на детальном обзоре SDXL, а сконцентрируемся именно на том, что авторы говорят о методике использования нескольких text encoders.

> Мы выбираем более мощный предварительно обученный кодировщик текста, который используем для обработки текста. В частности, мы используем открытый OpenCLIP ViT-bigG в сочетании с CLIP ViT-L, где объединяем предпоследние выходные данные текстового кодера вдоль оси канала.

Таким образом, объединение эмбеддингов происходит именно на уровне **предпоследнего слоя**, причем объединение идет **вдоль оси канала**.

> Помимо использования слоев перекрестного внимания для создания условий для модели при вводе текста, мы дополнительно создаем условия для модели при внедрении pooled текстовых эмбеддингов из модели OpenCLIP.

Следовательно, важно отметить, что используются именно pooled текстовые эмбеддинги.

**Вопрос.** А что такое pooled эмбеддинги?

<details>
  <summary><b>Ответ</b></summary>
  
  <font color='green'>Pooled эмбеддинг содержит информацию обо всем предложении сразу, а не о последовательности отдельных токенов.</font>
</details>

**Пример.** Представьте, что у вас есть текст, состоящий из 200 токенов, и вы хотите его обработать энкодером с максимальной длиной последовательности 512. В итоге вы получите последовательность из 200 эмбеддингов, каждый размерности 512. А эмбеддинг всего предложения, то есть pooled эмбеддинг, напротив, будет являться всего одним вектором размерности 512.

На данном семинаре мы не будем подробно останавливаться на том, какие есть методы пулинга, чтобы получить итоговый эмбеддинг предложения, однако упомянем несколько из них:
1. **CLS pooling.** Первый часто используемый метод пулинга заключается в использовании специального `<CLS>`-токена в начале каждого предложения. Он как раз и создается для того, чтобы улавливать информацию обо всем предложении сразу. Следовательно, слой пулинга заключается просто в том, что на его выходе выдается **эмбеддинг CLS-токена** и он используется как эмбеддинг всего предложения. (Например, в процессе обучения BERT такой CLS-токен использовался для предсказания следующего предложения, благодаря чему и обучался.)
2. **Mean pooling.** Второй опять же часто используемый метод — это пулинг усреднением. Как предполагается из названия, он аггрегирует информацию о предложении, производя усреднение эмбеддингов отдельных токенов. Аналогичные методы также могут использовать **max pooling** или **mean sqrt pooling**.

К сожалению, четкого ответа на вопрос о том, какой метод пулинга использовать, не существует, однако на HuggingFace **по умолчанию стоит именно CLS pooling**.

Вернемся к нашим нескольким текстовым энкодерам и рассмотрим то, как происходит процедура объединения их эмбеддингов в модели SXDL.

Как мы обсудили, по сути агрегация текстовой информации происходит в два этапа:
1. Итоговый текстовый эмбеддинг, который попадает на вход диффузионной модели, использует CLIP ViT-L & OpenCLIP ViT-bigG.
2. При этом помимо основного текстового эмбеддинга обуславливание на текстовую информацию происходит и за счет слоев перекрестного внимания (cross-attention), в которые попадают выходы OpenCLIP ViT-bigG после пулинга.

Рассмотрим реализацию каждого из пунктов в отдельности.

#### 3.1.1. Основной текстовый эмбеддинг на выходе двух энкодеров

Для того чтобы не загромождать наш ноутбук большим количеством кода, отсылаем вас к реализации `encode_prompt()` в [пайплайне SDXL](https://github.com/huggingface/diffusers/blob/c7617e482a522173ea6f922223aa010058552af8/src/diffusers/pipelines/stable_diffusion_xl/pipeline_stable_diffusion_xl.py#L214). А мы сконцентрируемся лишь на части этого метода.

```python
# в реализации можно задать два различных промпта, для каждого энкодера в отдельности
prompt = ...
prompt_2 = ...

# пропускаем промпты через токенайзеры (опять же отдельные для каждого энкодера)
text_inputs = tokenizer(
    prompt,
    padding="max_length",
    max_length=tokenizer.model_max_length,
    truncation=True,
    return_tensors="pt"
)
text_inputs_2 = tokenizer_2(
    prompt_2,
    padding="max_length",
    max_length=tokenizer_2.model_max_length,
    truncation=True,
    return_tensors="pt"
)

# получаем токенизированные входы для энкодеров
text_input_ids = text_inputs.ids
text_input_ids_2 = text_inputs_2.ids

# получаем эмбеддинги с помощью энкодеров
prompt_embeds = text_encoder(text_input_ids, output_hidden_states=True)
prompt_embeds_2 = text_encoder_2(text_input_ids_2, output_hidden_states=True)

# на выходе хотим иметь предпоследний слой
prompt_embeds = prompt_embeds.hidden_states[-2] # prompt_embeds.shape: torch.Size([1, 77, 768])
prompt_embeds_2 = prompt_embeds_2.hidden_states[-2] # prompt_embeds_2.shape: torch.Size([1, 77, 1280])

# конкатенируем эмбеддинги
prompt_embeds = torch.concat([prompt_embeds, prompt_embeds_2], dim=-1) # prompt_embeds.shape: torch.Size([1, 77, 2048])
```

Таким образом, видно, как исходные два текстовых эмбеддинга объединяются в один, причем для каждого токена идет объединение по соответствующей размерности эмбеддинга.

#### 3.1.2. Перекрестное внимание с текстовым эмбеддингом после пулинга

Давайте рассмотрим следующую простую реализацию, которую предложили сами авторы статьи SDXL.

Ранее мы еще не обсуждали следующие моменты, но сейчас столкнемся с дополнительным обуславливанием на:
1. Размер изображения
2. Выбор только части изображения (так называемый crop)
3. Соотношение сторон изображения

Каждое из этих условий также предобрабатывается заранее, а потом подается и конкатенируется вместе с текстовым эмбеддингом после пулинга.

In [ ]:
from einops import rearrange
import torch
import math

batch_size = 16
# channel dimension of pooled output of text encoder(s)
pooled_dim = 512

def fourier_embedding(inputs, outdim=256, max_period=10000):
    """
    Classical sinusoidal timestep embedding
    as commonly used in diffusion models
    :param inputs: batch of integer scalars shape [b,]
    :param outdim: embedding dimension
    :param max_period: max freq added
    :return: batch of embeddings of shape [b, outdim]
    """
    device = inputs.device
    half_dim = outdim // 2
    embeddings = math.log(max_period) / (half_dim - 1)
    embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
    embeddings = inputs[:, None] * embeddings[None, :]
    embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
    return embeddings

def cat_along_channel_dim(
        x:torch.Tensor,) -> torch.Tensor:
    if x.ndim == 1:
        x = x[...,None]
    assert x.ndim == 2
    b, d_in = x.shape
    x = rearrange(x, "b din -> (b din)")
    # fourier fn adds additional dimension
    emb = fourier_embedding(x)
    d_f = emb.shape[-1]
    emb = rearrange(emb, "(b din) df -> b (din df)",
                        b=b, din=d_in, df=d_f)
    return emb

def concat_embeddings(
        # batch of size and crop conditioning cf. Sec. 3.2
        c_size: torch.Tensor,
        c_crop: torch.Tensor,
        # batch of aspect ratio conditioning cf. Sec. 3.3
        c_ar: torch.Tensor,
        # final output of text encoders after pooling cf. Sec. 3.1
        c_pooled_txt: torch.Tensor, ) -> torch.Tensor:
    print('====> concat_embeddings()')
    # fourier feature for size conditioning
    c_size_emb = cat_along_channel_dim(c_size)
    print('c_size_emb.shape:', c_size_emb.shape)
    # fourier feature for crop conditioning
    c_crop_emb = cat_along_channel_dim(c_crop)
    print('c_crop_emb.shape:', c_crop_emb.shape)
    # fourier feature for aspect ratio conditioning
    c_ar_emb = cat_along_channel_dim(c_ar)
    print('c_ar_emb.shape:', c_ar_emb.shape)
    # the concatenated output is mapped to the same
    # channel dimension than the noise level conditioning
    # and added to that conditioning before being fed to the unet
    return torch.cat([c_pooled_txt,
                      c_size_emb,
                      c_crop_emb,
                      c_ar_emb], dim=1)

# simulating c_size as in Sec. 3.2
c_size = torch.zeros((batch_size, 2)).long()
print('c_size.shape:', c_size.shape)
# simulating c_crop as in Sec. 3.2
c_crop = torch.zeros((batch_size, 2)).long()
print('c_crop.shape:', c_crop.shape)
# simulating c_ar as in Sec. 3.3
c_ar = torch.zeros((batch_size, 2)).long()
print('c_ar.shape:', c_ar.shape)
# simulating pooled text encoder output as in Sec. 3.3
c_pooled = torch.zeros((batch_size, pooled_dim)).long()
print('c_pooled.shape:', c_pooled.shape)

# get concatenated embedding
c_concat = concat_embeddings(c_size, c_crop, c_ar, c_pooled)
print('c_concat.shape:', c_concat.shape)

Таким образом, на выходе получаем эмбеддинг размера 2048, который объединяет в себе сразу всю информацию об исходном промпте и наборе параметров генерации.

### 3.2. Stable Diffusion 3

А теперь давайте перейдем к более современной модели — [Stable Diffusion 3](https://stabilityai-public-packages.s3.us-west-2.amazonaws.com/Stable+Diffusion+3+Paper.pdf).

Она так же, как и SDXL, использует несколько текстовых энкодеров, однако делает это немного иначе. Опять же, сейчас мы не будем вдаваться в подробности архитектуры (спойлер: мы это обсудим в следующем разделе), а сконцентрируемся только на теме текстовых эмбеддингов.

Итак, в начале статьи авторы упоминают, что идея использования нескольких текстовых энкодеров различного сорта (не CLIP и OpenCLIP, как в SDXL, а принципиально разных) уже получила свое развитие в работе [Balaji et al. 2023 *eDiff-I: Text-to-Image Diffusion Models with an Ensemble of Expert Denoisers*](https://arxiv.org/pdf/2211.01324).

**Вопрос.** Как вы думаете: в чем может быть смысл использовать принципиально разные текстовые энкодеры, например CLIP и T5?

<details>
  <summary><b>Ответ</b></summary>
  
  <font color='green'>Каждый из энкодеров имеет уникальную архитектуру и обучался на своем наборе данных, благодаря чему какой-то может обеспечивать лучшую композицию кадра, а какой-то лучше понимает стиль и детали.</font>
</details>

Однако помимо базовой идеи с различным пониманием текста у энкодеров есть и другая причина использовать несколько разных моделей. Авторы Stable Diffusion 3 во время обучения с вероятностью $46.4\%$ выкидывают каждый из текстовых энкодеров (буквально зануляют его выходы, производя тем самым drop-out). Как они утверждают (и на самом деле доказывают экспериментально), это позволяет уже на инференсе использовать только один из энкодеров, более легковесный.

**Вопрос.** Как вы думаете: почему именно $46.4\%$ — шанс зануления каждого из эмбеддингов?

**Ответ.** На самом деле все просто — в таком случае в процессе обучения модель учится без текстового условия примерно в $10\%$ случаев, что является рекомендуемым значением в обучении с classifier-free guidance подходом.

Если переходить к конкретике, авторы используют комбинацию из трех различных энкодеров: OpenCLIP-bigG/14, CLIP-L/14 и T5 XXL.

#### 3.2.1. Архитектура

Перейдем к архитектуре, а именно к той ее части, которая связана с текстовыми энкодерами. Рассмотрим следующую схемку из оригинальной статьи Stable Diffusion 3.

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1Cb17ztm0Bjg0DLGVmjzKbOUU1KPOgk0x" alt="sd3-text" width="500"/>
    <figcaption> Использование трех текстовых энкодеров в модели Stable Diffusion 3. Pooled эмбеддинги (левая часть) от энкодеров CLIP используются в слоях перекрестного внимания (cross-attention). Конкатенированные эмбеддинги от всех трех энкодеров подаются как контекст на вход модели. Источник: <a href="https://stabilityai-public-packages.s3.us-west-2.amazonaws.com/Stable+Diffusion+3+Paper.pdf">Esser et al. 2024</a> </figcaption>
</figure>

Аналогично тому, как это происходило в SDXL, процесс обработки текстовой информации (caption) состоит из нескольких частей:
1. Итоговый текстовый эмбеддинг является конкатенацией трех эмбеддингов от отдельных замороженных моделей: OpenCLIP-bigG/14, CLIP-L/14 и T5 XXL.
2. В слои перекрестного внимания (cross-attention) подаются эмбеддинги после пулинга, полученные конкатенацией от двух моделей CLIP.

То есть, опять же,
1. (Справа на картинке) Конкатенация предпоследних скрытых представлений вдоль оси канала: $c_{\text{ctxt}}^{\text{CLIP-G/14}} \in \mathbb{R}^{77 \times 1280}$ и $c_{\text{ctxt}}^{\text{CLIP-L/14}} \in \mathbb{R}^{77 \times 768}$ объединяются в $c_{\text{ctxt}}^{\text{CLIP}} \in \mathbb{R}^{77 \times 2048}$, а затем и с $c_{\text{ctxt}}^{\text{T5}} \in \mathbb{R}^{77 \times 4096}$, при этом добиваются нулями до размерности 4096 вдоль каждого токена. Итак, финальный контекст на вход модели есть $c_{\text{ctxt}} \in \mathbb{R}^{154 \times 4096}$.
2. (Слева на картинке) Конкатенация pooled-выходов размеров $768$ и $1280$ от CLIP-L/14 и OpenCLIP-bigG/14 соответственно.

#### 3.2.2. Пример влияния отключения T5-энкодера на качество генерации

Рассмотрим, как сказывается отбрасывание T5 XXL текстового энкодера на генерации.

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=179dSuEpwIQfQvOl8Zxd0papvPBGLbdWv" alt="text-encoder-drop" width="500"/>
    <figcaption> <b>Влияние T5-энкодера.</b> Можно наблюдать, что T5 более важен для сложных промптов, в частности для качественной детализации. Однако для большинства промптов удаление T5 на инференсе все же дает существенное ускорение. Источник: <a href="https://stabilityai-public-packages.s3.us-west-2.amazonaws.com/Stable+Diffusion+3+Paper.pdf">Esser et al. 2024</a> </figcaption>
</figure>

Таким образом, по большому счету прирост в качестве действительно есть, но только в сложных композициях, требующих хорошего понимания текстового запроса.

Тем не менее для большинства промптов будет хватать обычного CLIP-энкодера.

## 4. DiT и реализация MMDiT из SD3

Перейдем к последней на сегодня очень интересной теме — использованию трансформерных архитектур в диффузионных моделях.

Напомним, что классическим решением для предсказания шума в DDPM-подобных моделях было использование U-Net.

И наверняка вы могли задаваться вопросом: почему именно U-Net, нет ли возможности использовать что-то еще?

Так вот сейчас мы обсудим другую модель, фундаментом которой стал Vision Transformer (ViT), а именно **Diffusion Transformer (DiT)**, предложенную в [этой статье](https://arxiv.org/abs/2212.09748).

### 4.1. Напоминание о Vision Transformer (ViT)

Для того чтобы погрузиться в реализацию DiT, сначала вспомним базовые концепции и идеи из Vision Transformer.

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1onA7x-72bmLnCnOKxnog8g0eicn2-4FE" alt="vit" width="800"/>
    <figcaption> <b>Архитектура модели Vision Transformer (ViT).</b> Изображение разрезается на патчи фиксированного размера, затем они линейно преобрауются в эмбеддинги, к ним добавляются позиционные эмбеддинги, а после этого результат подается в виде векторов в стандартный трансформерный энкодер. Для задачи классификации также обучается специальный `[class] embedding`. Источник: <a href="https://arxiv.org/pdf/2010.11929">Dosovitskiy et al. 2021</a> </figcaption>
</figure>

1. ViT состоит из последовательных трансформерных энкодерных блоков — **Transformer Encoder Layer**. Эти слои работают с последовательностью визуальных токенов исходного изображения.
2. Эти визуальные токены — части исходного изображения, которые выглядят как отдельные квадраты (патчи) в сетке на изображении (обычно порядка $16 \times 16$ патчей на изображении).
3. Совместно с этими визуальными токенами на вход трансформерных блоков также подается и специальный обучаемый **`<CLS>`-токен**. Когда ViT обучается для задачи классификации изображений, этот `<CLS>`-токен подается на вход итоговой MLP головы на выходе сети.
4. Для преобразования исходного изображения в последовательность патчей внутри ViT есть специальный блок — **Patch Embedding Block** (слева на картинке). Внутри него патчи с помощью линейного слоя преобразуются в векторы, к которым затем добавляются позиционные эмбеддинги (классические синусоидальные).
5. Архитектура трансформерного слоя достаточно классическая (справа на картинке) и включает в себя последовательность из LayerNorm, Multi-Head Attention, LayerNorm и MLP. Отметим, что также используются **два слоя skip connection**.

### 4.2. Реализация DiT

Теперь перейдем к тому, как авторы DiT решили изменить архитектуру ViT, чтобы предсказывать добавляемый к изображению шум (на самом деле это все происходит в латентном пространстве, не стоит об этом забывать, но для простоты изложения думаем об этом как о реальных изображениях).

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=1kuVMWAN2YwEKYsnJVyhNgRvhuMac4Ywz" alt="dit" width="800"/>
    <figcaption> <b>Архитектура модели Diffusion Transformer (DiT).</b> Слева: Входная латентная информация разбивается на патчи и обрабатывается несколькими блоками DiT. Справа: Подробная информация о блоках DiT. Эксперименты с вариантами стандартных блоков-трансформеров (adaLN-Zero, Cross-Attention, In-Context Conditioning) привели к тому, что первый работает лучше всего. Источник: <a href="https://arxiv.org/abs/2212.09748">Peebles et al. 2023</a> </figcaption>
</figure>

1. Первое, на что мы обратим внимание, — модель DiT является **условной**, она принимает на вход помимо исходного Noised Latent также **Embed**, который аггрегирует в себе информацию о
*   классовой метке изображения $y$,
*   временном шаге $t$.

2. Вторая особенность заключается в использовании специального **Adaptive LayerNorm-Zero** блока. Похожая идея использовалась еще в архитектуре [StyleGAN](https://arxiv.org/abs/1812.04948), где авторы предлагали Adaptive InstanceNorm.

   - Сам по себе Adaprive LayerNorm предполагает обучение двух параметров $\gamma$ и $\beta$, которые отвечают за умножение и сдвиг. Вместо того, чтобы просто напрямую их учить, в DiT предлагается получать их с помощью MLP из совместного эмбеддинга Embed (из label и timestep).
   - Важный момент состоит в приписке **-Zero**, которая говорит о том, что авторы инициализируют MLP нулями.
   - **Вопрос.** Как при этом тогда выглядит выход всего блока DiT?
   - **Ответ.** Он совпадает со входом, поскольку в архитектуре есть слои skip connection.

Итак, мы обсудили, чем отличается DiT от классического ViT.

**Вопрос (с подвохом).** А где мы используем текстовую информацию (caption), чтобы построить диффузионную модель text2image?

<details>
  <summary><b>Ответ</b></summary>
  
  <font color='green'>DiT не предполагает использование текстового контекста, а обуславливается лишь на метку класса и временной шаг. Для того чтобы использовать DiT для text2image-генерации, нужно изменить архитектуру обработки контекста.

<b>Примечание.</b> Конечно, можно зашить текстовый эмбеддинг в качестве **Embed** и таким образом подавать его в модель, и такие варианты есть. Однако есть вариант и получше :)</font>
</details>

Таким образом, мы плавно пришли к тому, что сейчас активно используется в последних диффузионных моделях — архитектуре MM-DiT.

### 4.3. Multimodal DiT (MM-DiT)

Авторы [Stable Diffusion 3](https://stabilityai-public-packages.s3.us-west-2.amazonaws.com/Stable+Diffusion+3+Paper.pdf) построили свою архитектуру на основе DiT.

Аналогично DiT, они используют эмбеддинги временного шага $t$ и $c_{\text{vec}}$ как входы адаптивных слоев нормализации (или, как они пишут, механизмов модуляции).

Важный момент заключается в том, что такого обуславливания критически не хватает для полноценной text2image-генерации, ведь pooled текстовые эмбеддинги содержат только обобщенную информацию о предложении.

Таким образом, авторы предлагают использовать полноценный текстовый контекст как дополнительный вход каждого блока.

<figure align="center">
    <img src="https://drive.google.com/uc?export=view&id=10b1ZU9FwtOVI1FQO8YZCSYGJmWd5s1Pp" alt="mmdit" width="1000"/>
    <figcaption> <b>Архитектура модели Multimodal Diffusion Transformer (MM-DiT).</b> Слева: полная архитектура со всеми компонентами. Справа: один MM-DiT блок. Источник: <a href="https://arxiv.org/abs/2212.09748">Peebles et al. 2023</a> </figcaption>
</figure>

Давайте последовательно разбираться с тем, что происходит с текстовой и визуальной информацией в этой модели.

**Текст**

1. На вход модели приходит **Caption**, то есть текстовое описание, по которому мы хотим сгенерировать изображение.
2. Этот **Caption** проходит через три различных текстовых энкодера (как обсуждалось ранее), при этом получаются pooled текстовый эмбеддинг $c_{\text{vec}} \in \mathbb{R}^{2048}$ и полноценный текстовый контекст $c_{\text{ctxt}} \in \mathbb{R}^{154 \times 4096}$.
3. Pooled текстовый эмбеддинг $c_{\text{vec}}$ соединяется с векторным представлением временного шага **Timestep**, который уже заранее предобрабатывается с позиционным эмбеддингом. В итоге получается $y$.
4. Полноценный текстовый контекст $c_{\text{ctxt}}$ проходит через линейный слой, в итоге реализуя контекст $c$.
   
**Изображение**

1. Зашумленный латент **Noised Latent** разделяется на патчи и пропускается через линейный слой, совмещаясь с позиционными эмбеддингами, как это было и в DiT. В итоге имеем $x$.

**Вопрос.** А что будет происходить дальше, судя по схеме?

**Ответ.** Текстовая и визуальная информация **по отдельности** проходят через трансформерные блоки MM-DiT, взаимодействуя **только** посредством механизма **Attention**.

Да, на первый взгляд такая архитектура кажется ужасным усложнением, ведь мы буквально дублируем исходную архитектуру DiT и повторяем ее дважды. Однако именно это позволяет нам полноценно учитывать текстовую информацию.

### 4.4. Имплементация одного блока MM-DiT

А теперь давайте перейдем к реализации одного блока MM-DiT.

In [ ]:
# pip install x_transformers
from typing import Tuple

import torch
from torch import nn
from torch import Tensor
import torch.nn.functional as F
from torch.nn import Module, ModuleList

from einops import rearrange, repeat, pack, unpack
from einops.layers.torch import Rearrange

from x_transformers.attend import Attend
from x_transformers import (
    RMSNorm,
    FeedForward
)

# Вспомогательные функции

def exists(v):
    return v is not None

def default(v, d):
    return v if exists(v) else d

def softclamp(t, value):
    return (t / value).tanh() * value

# Слой нормировки

class MultiHeadRMSNorm(Module):
    def __init__(self, dim, heads = 1):
        super().__init__()
        self.scale = dim ** 0.5
        self.gamma = nn.Parameter(torch.ones(heads, 1, dim))

    def forward(self, x):
        return F.normalize(x, dim = -1) * self.gamma * self.scale

# Слой совместного внимания текста и изображения

class JointAttention(Module):
    def __init__(
        self,
        *,
        dim,
        dim_inputs: Tuple[int, ...],
        dim_head = 64,
        heads = 8,
        qk_rmsnorm = False,
        flash = False,
        softclamp = False,
        softclamp_value = 50.,
        attend_kwargs: dict = dict()
    ):
        super().__init__()
        """
        ein notation

        b - batch
        h - heads
        n - sequence
        d - feature dimension
        """

        dim_inner = dim_head * heads

        num_inputs = len(dim_inputs)
        self.num_inputs = num_inputs

        self.to_qkv = ModuleList([nn.Linear(dim_input, dim_inner * 3, bias = False) for dim_input in dim_inputs])

        self.split_heads = Rearrange('b n (qkv h d) -> qkv b h n d', h = heads, qkv = 3)

        self.attend = Attend(
            flash = flash,
            softclamp_logits = softclamp,
            logit_softclamp_value = softclamp_value,
            **attend_kwargs
        )

        self.merge_heads = Rearrange('b h n d -> b n (h d)')

        self.to_out = ModuleList([nn.Linear(dim_inner, dim_input, bias = False) for dim_input in dim_inputs])

        self.qk_rmsnorm = qk_rmsnorm
        self.q_rmsnorms = (None,) * num_inputs
        self.k_rmsnorms = (None,) * num_inputs

        if qk_rmsnorm:
            self.q_rmsnorms = ModuleList([MultiHeadRMSNorm(dim_head, heads = heads) for _ in range(num_inputs)])
            self.k_rmsnorms = ModuleList([MultiHeadRMSNorm(dim_head, heads = heads) for _ in range(num_inputs)])

        self.register_buffer('dummy', torch.tensor(0), persistent = False)

    def forward(
        self,
        inputs: Tuple[Tensor],
        masks: Tuple[Tensor | None] | None = None
    ):

        device = self.dummy.device

        assert len(inputs) == self.num_inputs

        masks = default(masks, (None,) * self.num_inputs)

        # Проецируем каждую модальность отдельно на qkv

        all_qkvs = []
        all_masks = []

        for x, mask, to_qkv, q_rmsnorm, k_rmsnorm in zip(inputs, masks, self.to_qkv, self.q_rmsnorms, self.k_rmsnorms):

            qkv = to_qkv(x)
            qkv = self.split_heads(qkv)

            # Опционально делаем отдельно нормировку qk для каждой модальности

            if self.qk_rmsnorm:
                q, k, v = qkv
                q = q_rmsnorm(q)
                k = k_rmsnorm(k)
                qkv = torch.stack((q, k, v))

            all_qkvs.append(qkv)

            # Оперируем масками для каждой модальности

            if not exists(mask):
                mask = torch.ones(x.shape[:2], device = device, dtype = torch.bool)

            all_masks.append(mask)

        # Комбинируем все qkv и маски

        all_qkvs, packed_shape = pack(all_qkvs, 'qkv b h * d')
        all_masks, _ = pack(all_masks, 'b *')

        # Разделяем qkv на части

        q, k, v = all_qkvs

        outs, *_ = self.attend(q, k, v, mask = all_masks)

        # Объединяем отдельные головы

        outs = self.merge_heads(outs)
        outs = unpack(outs, packed_shape, 'b * d')

        # Отделяем комбинации голов для каждой модальности

        all_outs = []

        for out, to_out in zip(outs, self.to_out):
            out = to_out(out)
            all_outs.append(out)

        return tuple(all_outs)

# Класс одного блока MM-DiT

class MMDiTBlock(Module):
    def __init__(
        self,
        *,
        dim_joint_attn,
        dim_text,
        dim_image,
        dim_cond = None,
        dim_head = 64,
        heads = 8,
        qk_rmsnorm = False,
        flash_attn = False,
        ff_kwargs: dict = dict()
    ):
        super().__init__()

        # Обуславливание на временной шаг

        has_cond = exists(dim_cond)
        self.has_cond = has_cond

        if has_cond:
            dim_gammas = (
                *((dim_text,) * 4),
                *((dim_image,) * 4)
            )

            dim_betas = (
                *((dim_text,) * 2),
                *((dim_image,) * 2),
            )

            self.cond_dims = (*dim_gammas, *dim_betas)

            to_cond_linear = nn.Linear(dim_cond, sum(self.cond_dims))

            self.to_cond = nn.Sequential(
                Rearrange('b d -> b 1 d'),
                nn.SiLU(),
                to_cond_linear
            )

            nn.init.zeros_(to_cond_linear.weight)
            nn.init.zeros_(to_cond_linear.bias)
            nn.init.constant_(to_cond_linear.bias[:sum(dim_gammas)], 1.)

        # Адаптивная нормализация

        self.text_attn_layernorm = nn.LayerNorm(dim_text, elementwise_affine = not has_cond)
        self.image_attn_layernorm = nn.LayerNorm(dim_image, elementwise_affine = not has_cond)

        self.text_ff_layernorm = nn.LayerNorm(dim_text, elementwise_affine = not has_cond)
        self.image_ff_layernorm = nn.LayerNorm(dim_image, elementwise_affine = not has_cond)

        # Attention и FeedForward

        self.joint_attn = JointAttention(
            dim = dim_joint_attn,
            dim_inputs = (dim_text, dim_image),
            dim_head = dim_head,
            heads = heads,
            flash = flash_attn
        )

        self.text_ff = FeedForward(dim_text, **ff_kwargs)
        self.image_ff = FeedForward(dim_image, **ff_kwargs)

    def forward(
        self,
        *,
        text_tokens,
        image_tokens,
        text_mask = None,
        time_cond = None,
        skip_feedforward_text_tokens = True
    ):
        assert not (exists(time_cond) ^ self.has_cond), 'time condition must be passed in if dim_cond is set at init. it should not be passed in if not set'

        if self.has_cond:
            (
                text_pre_attn_gamma,
                text_post_attn_gamma,
                text_pre_ff_gamma,
                text_post_ff_gamma,
                image_pre_attn_gamma,
                image_post_attn_gamma,
                image_pre_ff_gamma,
                image_post_ff_gamma,
                text_pre_attn_beta,
                text_pre_ff_beta,
                image_pre_attn_beta,
                image_pre_ff_beta,
            ) = self.to_cond(time_cond).split(self.cond_dims, dim = -1)

        # Attention адаптивная нормализация

        text_tokens_residual = text_tokens
        image_tokens_residual = image_tokens

        text_tokens = self.text_attn_layernorm(text_tokens)
        image_tokens = self.image_attn_layernorm(image_tokens)

        if self.has_cond:
            text_tokens = text_tokens * text_pre_attn_gamma + text_pre_attn_beta
            image_tokens = image_tokens * image_pre_attn_gamma + image_pre_attn_beta

        # Attention

        text_tokens, image_tokens = self.joint_attn(
            inputs = (text_tokens, image_tokens),
            masks = (text_mask, None)
        )

        # Обусловленный attention выход

        if self.has_cond:
            text_tokens = text_tokens * text_post_attn_gamma
            image_tokens = image_tokens * image_post_attn_gamma

        # Добавляем attention residual

        text_tokens = text_tokens + text_tokens_residual
        image_tokens = image_tokens + image_tokens_residual

        # FeedForward адаптивная нормализация

        text_tokens_residual = text_tokens
        image_tokens_residual = image_tokens

        text_tokens = self.text_attn_layernorm(text_tokens)
        image_tokens = self.image_attn_layernorm(image_tokens)

        if self.has_cond:
            text_tokens = text_tokens * text_pre_ff_gamma + text_pre_ff_beta
            image_tokens = image_tokens * image_pre_ff_gamma + image_pre_ff_beta

        # FeedForward на изображениях

        image_tokens = self.image_ff(image_tokens)

        # Картиночный обусловленный attention выход

        if self.has_cond:
            image_tokens = image_tokens * image_post_ff_gamma

        # Добавляем картиночный attention residual

        image_tokens = image_tokens + image_tokens_residual

        # Преждевременный выход, без последнего слоя

        if skip_feedforward_text_tokens:
            return text_tokens, image_tokens

        # FeedForward для текста

        text_tokens = self.text_ff(text_tokens)

        # Текстовый обусловленный attention выход

        if self.has_cond:
            text_tokens = text_tokens * text_post_ff_gamma

        # Добавляем текстовый attention residual

        text_tokens = text_tokens + text_tokens_residual

        return text_tokens, image_tokens

In [ ]:
import torch

# Инициализируем один блок MM-DiT

block = MMDiTBlock(
    dim_joint_attn=512,
    dim_cond=256,
    dim_text=768,
    dim_image=512,
    qk_rmsnorm=True
)

# Инициализируем входные тензоры

time_cond = torch.randn(2, 256)

text_tokens = torch.randn(2, 512, 768)
text_mask = torch.ones((2, 512)).bool()

image_tokens = torch.randn(2, 1024, 512)

# Пропускаем через блок MM-DiT

text_tokens_next, image_tokens_next = block(
    time_cond=time_cond,
    text_tokens=text_tokens,
    text_mask=text_mask,
    image_tokens=image_tokens
)

print('text_tokens_next.shape:', text_tokens_next.shape)
print('image_tokens_next.shape:', image_tokens_next.shape)

Таким образом, мы с Вами реализовали один блок из архитектуры MM-DiT.

## Итоги

Подведем итоги нашего сегодняшнего занятия.
1. Модель DDPM слишком медленно работает на инференсе, одним из решений является DDIM, в котором прямой процесс перестает быть марковским. При этом функция потерь в DDIM совпадает с той же в DDPM, что позволяет обучить предсказатель шума в парадигме DDPM, а затем использовать DDIM для ускоренного инференса. Ускоренный процесс сэмплирования достигается за счет пропуска шагов в обратном процессе и имеет тем меньшую погрешность, чем более детерминистичный процесс используется. Именно поэтому DDIM оказывается удачным решением для ускорения.
2. Помимо DDIM существует множество других «планировщиков шума», которые выполняют функцию зашумления в прямом процессе и функцию расшумления в обратном. Глобально их можно разделить на детерминистические и стохастические. Выбор планировщика шума для определенной задачи остается за вами, но мы рекомендуем присмотреться к [`DPMSolverMultistepScheduler`](https://huggingface.co/docs/diffusers/api/schedulers/multistep_dpm_solver) и [`EulerDiscreteScheduler`](https://huggingface.co/docs/diffusers/api/schedulers/euler).
3. Для повышения согласованности сгенерированного изображения заданной текстовой инструкции в последних диффузионных моделях часто используют несколько текстовых энкодеров. Так, в модели **SDXL** применяют два энкодера: CLIP ViT-L и OpenCLIP-bigG, а в **Stable Diffusion 3** уже три: CLIP ViT-L, OpenCLIP-bigG и T5 XXL.
4. Несмотря на то, что классическим выбором архитектуры для предсказания шума исконно являлся U-Net, последние диффузионные модели используют трансформерные архитектуры на основе ViT. Первая такая модель (DiT) обуславливалась на метку класса, но для text-to-image генерации этого оказалось мало. В Stable Diffusion 3 предложили новую архитектуру, которая независимыми трансформерными блоками обрабатывает текстовую и визуальную информацию, пересекая их только лишь на слое перекрестного внимания.

### Полезные источники

Планировщики шума:
- https://blog.segmind.com/what-are-schedulers-in-stable-diffusion/
- https://huggingface.co/docs/diffusers/api/schedulers/overview
- https://stable-diffusion-art.com/samplers/
- https://www.felixsanz.dev/articles/complete-guide-to-samplers-in-stable-diffusion
- https://jarvislabs.ai/docs/samplers
- https://civitai.com/articles/7484/understanding-stable-diffusion-samplers-beyond-image-comparisons

Несколько текстовых энкодеров (на примере SDXL и SD3):
- https://discuss.huggingface.co/t/sdxl-custom-pipeline-input-to-unet-why-2-text-encoders/49731/4
- https://arxiv.org/abs/2307.01952
- https://stabilityai-public-packages.s3.us-west-2.amazonaws.com/Stable+Diffusion+3+Paper.pdf
- https://arxiv.org/pdf/2211.01324

Трансформерные архитектуры в диффузионных моделях (на примере DiT и SD3):
- https://arxiv.org/abs/2212.09748
- https://github.com/facebookresearch/dit
- https://medium.com/@threehappyer/understanding-dit-diffusion-transformer-in-one-article-2f7c330ad0ea
- https://huggingface.co/docs/diffusers/api/pipelines/dit
- https://github.com/lucidrains/mmdit?tab=readme-ov-file
- https://encord.com/blog/stable-diffusion-3-text-to-image-model/
- https://youtu.be/aSLDXdc2hkk?si=hG45HC8HCVGCWafb